## VD+ Pedidos

In [ ]:
# Script para processar múltiplos arquivos e atualizar dados com formatação
import pandas as pd
import os
import glob
from openpyxl import load_workbook
from openpyxl.styles import numbers
import logging
import shutil # Apenas se for usar a opção de mover arquivos
# Configura mensagens de log
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Configurações ---
caminho_downloads = r'C:\Users\Florestas\Downloads'
caminho_destino_dados = r'C:\Users\Florestas\Desktop\Relatorio Oficial 2025\Automação\Dados'
# Pasta para mover arquivos processados (opcional)
# caminho_arquivados = os.path.join(caminho_destino_dados, 'Arquivados')
nome_arquivo_destino = 'consultapedidos.xlsx'
caminho_completo_destino = os.path.join(caminho_destino_dados, nome_arquivo_destino)
padrao_arquivo_origem = '*consultapedidos*.xlsx'

colunas_desejadas = [
    'CodigoPedido', 'SituaçãoFiscal', 'NotaFiscal', 'QtdeMateriais', 'QtdeItens', 'ValorLiquido', 'ValorPedido',
    'MeioCaptacao', 'Tipo de Entrega', 'SituaçãoComercial', 'DetalheSituaçãoComercial',
    'Data Captação', 'Data Aprovação', 'DataFaturamento', 'DataEntrega', 'PrevisãoEntrega',
    'Cidade', 'CEP', 'CidadeEntregaRetirada', 'CEPEntregaRetirada', 'Cód Externo Pedido'
]
colunas_valores = ['ValorLiquido', 'ValorPedido']

# --- Funções Auxiliares ---
def formatar_valores_excel(caminho, colunas_formatar):
    logging.info(f"Aplicando formatação monetária R$ em '{caminho}'...")
    try:
        wb = load_workbook(caminho)
        ws = wb.active
        header = {cell.value: idx for idx, cell in enumerate(ws[1], start=1)}
        cols_para_formatar_idx = []
        for col_nome in colunas_formatar:
             if col_nome in header:
                  cols_para_formatar_idx.append(header[col_nome])
             else:
                  logging.warning(f"Coluna '{col_nome}' para formatação não encontrada no cabeçalho de '{caminho}'.")
        if not cols_para_formatar_idx:
             logging.warning("Nenhuma coluna válida encontrada para aplicar formatação monetária.")
             return
        for col_idx in cols_para_formatar_idx:
            for linha in ws.iter_rows(min_row=2, min_col=col_idx, max_col=col_idx):
                for celula in linha:
                    if isinstance(celula.value, (int, float)):
                         celula.number_format = 'R$ #,##0.00'
        wb.save(caminho)
        logging.info(f"Formatação monetária aplicada com sucesso.")
    except FileNotFoundError:
        logging.error(f"Erro ao formatar: Arquivo '{caminho}' não encontrado.")
    except Exception as e:
        logging.error(f"Erro inesperado ao formatar Excel '{caminho}': {e}")

def safe_to_numeric(series):
    series_str = series.astype(str).str.replace(',', '.', regex=False)
    series_str = series_str.str.replace(r'[^\d.]', '', regex=True)
    return pd.to_numeric(series_str, errors='coerce')

# --- Processo Principal ---
logging.info(f"Iniciando processo de atualização de '{nome_arquivo_destino}'...")
if not os.path.exists(caminho_destino_dados):
    logging.error(f"A pasta de destino '{caminho_destino_dados}' não existe. Processo interrompido.")
else:
    caminho_busca = os.path.join(caminho_downloads, padrao_arquivo_origem)
    arquivos_encontrados = glob.glob(caminho_busca)
    if not arquivos_encontrados:
        logging.warning(f"Nenhum arquivo correspondente a '{padrao_arquivo_origem}' foi encontrado em '{caminho_downloads}'.")
    else:
        logging.info(f"Arquivos encontrados em Downloads: {len(arquivos_encontrados)}")
        for f in arquivos_encontrados: logging.info(f"  -> {os.path.basename(f)}")
        lista_novos_dados = []
        arquivos_processados_com_sucesso = []
        for caminho_arquivo_origem in arquivos_encontrados:
            try:
                nome_base_arquivo = os.path.basename(caminho_arquivo_origem)
                logging.info(f"Lendo arquivo: '{nome_base_arquivo}'")
                df_novo = pd.read_excel(caminho_arquivo_origem, dtype=str)
                colunas_existentes_novo = [col for col in colunas_desejadas if col in df_novo.columns]
                df_novo = df_novo[colunas_existentes_novo]
                for col in colunas_valores:
                    if col in df_novo.columns: df_novo[col] = safe_to_numeric(df_novo[col])
                    else: logging.warning(f"Coluna de valor '{col}' não encontrada em '{nome_base_arquivo}'.")
                if 'CodigoPedido' not in df_novo.columns:
                     logging.error(f"Coluna 'CodigoPedido' essencial não encontrada em '{nome_base_arquivo}'. Arquivo será ignorado.")
                     continue
                lista_novos_dados.append(df_novo)
                arquivos_processados_com_sucesso.append(caminho_arquivo_origem)
                logging.info(f"Arquivo '{nome_base_arquivo}' processado.")
            except Exception as e:
                logging.error(f"Erro ao processar o arquivo '{nome_base_arquivo}': {e}")
        if not lista_novos_dados:
            logging.warning("Nenhum arquivo novo pôde ser lido e processado com sucesso.")
        else:
            novos_dados_combinados = pd.concat(lista_novos_dados, ignore_index=True)
            logging.info(f"Total de {len(novos_dados_combinados)} linhas lidas dos arquivos novos.")
            df_final = pd.DataFrame()
            if os.path.exists(caminho_completo_destino):
                logging.info(f"Arquivo de destino '{nome_arquivo_destino}' existente encontrado. Lendo dados antigos...")
                try:
                    dados_existentes = pd.read_excel(caminho_completo_destino, dtype=str)
                    colunas_existentes_antigo = [col for col in colunas_desejadas if col in dados_existentes.columns]
                    dados_existentes = dados_existentes[colunas_existentes_antigo]
                    for col in colunas_valores:
                        if col in dados_existentes.columns: dados_existentes[col] = safe_to_numeric(dados_existentes[col])
                        else: logging.warning(f"Coluna de valor '{col}' não encontrada no arquivo existente.")
                    if 'CodigoPedido' not in dados_existentes.columns:
                         logging.error(f"Coluna 'CodigoPedido' não encontrada no arquivo existente '{nome_arquivo_destino}'. Usando apenas os dados novos.")
                         df_final = novos_dados_combinados
                    else:
                         logging.info("Combinando dados existentes com os novos...")
                         df_combinado_temp = pd.concat([dados_existentes, novos_dados_combinados], ignore_index=True)
                         logging.info(f"Atualizando/Adicionando registros baseado no 'CodigoPedido'...")
                         df_final = df_combinado_temp.drop_duplicates(subset='CodigoPedido', keep='last')
                         logging.info(f"Total de linhas após combinação e atualização: {len(df_final)}")
                except Exception as e:
                    logging.error(f"Erro ao ler ou processar o arquivo existente '{nome_arquivo_destino}': {e}. Considerando apenas dados novos.")
                    df_final = novos_dados_combinados
            else:
                logging.info(f"Arquivo de destino '{nome_arquivo_destino}' não encontrado. Usando apenas dados novos.")
                df_final = novos_dados_combinados
            if not df_final.empty:
                 if 'CodigoPedido' not in df_final.columns:
                      logging.error("Coluna 'CodigoPedido' está faltando no DataFrame final. Não foi possível salvar.")
                 else:
                    try:
                        logging.info(f"Salvando dados finais em '{caminho_completo_destino}'...")
                        for col in colunas_desejadas:
                            if col not in df_final.columns: df_final[col] = pd.NA
                        df_final = df_final[colunas_desejadas]
                        df_final.to_excel(caminho_completo_destino, index=False)
                        logging.info(f"✅ Dados salvos com sucesso em '{nome_arquivo_destino}'.")
                        formatar_valores_excel(caminho_completo_destino, colunas_valores)
                    except Exception as e:
                        logging.error(f"Erro ao salvar o arquivo final '{nome_arquivo_destino}': {e}")
            else:
                 logging.warning("Nenhum dado final para salvar (verifique os arquivos de origem).")
logging.info("Processo concluído.")


import os
import pandas as pd
import csv
import unicodedata
import re
from datetime import datetime

# Caminhos
caminho_destino = r'C:\Users\Florestas\Desktop\Relatorio Oficial 2025\Automação\Dados'
nome_padrao_destino = 'consultapedidos.xlsx'
caminho_arquivo_origem = os.path.join(caminho_destino, nome_padrao_destino)
caminho_csv_teste = os.path.join(caminho_destino, 'consultapedidosvd.csv')

# Função para limpar os nomes das colunas
def limpar_nome_coluna(col):
    if not isinstance(col, str):
        return col
    col = unicodedata.normalize('NFKD', col).encode('ASCII', 'ignore').decode('utf-8')
    col = re.sub(r'[^\w]', '', col)
    return col

# Função para converter datas para o formato yyyy-mm-dd
def converter_data_para_iso(valor):
    if isinstance(valor, str):
        # Tenta detectar datas no formato dd/mm/yyyy ou dd-mm-yyyy
        match = re.match(r'^(\d{2})[/-](\d{2})[/-](\d{4})$', valor.strip())
        if match:
            dia, mes, ano = match.groups()
            try:
                return f"{ano}-{mes}-{dia}"
            except:
                return valor
    return valor

if os.path.exists(caminho_arquivo_origem):
    dados = pd.read_excel(caminho_arquivo_origem, dtype=str)

    # Limpa os nomes das colunas
    dados.columns = [limpar_nome_coluna(col) for col in dados.columns]

    # Remove quebras de linha e espaços extras
    dados = dados.applymap(lambda x: str(x).replace('\n', ' ').strip() if pd.notna(x) else '')

    # Remove horas de colunas com datas
    for coluna in dados.columns:
        if any(keyword in coluna.lower() for keyword in ['data', 'dt']):
            # Remove horas do tipo "dd/mm/yyyy hh:mm"
            dados[coluna] = dados[coluna].apply(
                lambda x: re.sub(r'\s+\d{2}:\d{2}(:\d{2})?$', '', x) if isinstance(x, str) else x
            )
            # Converte para o formato yyyy-mm-dd
            dados[coluna] = dados[coluna].apply(converter_data_para_iso)

    # Substitui 'nan' por string vazia em colunas conhecidas
    colunas_para_validar = ['DataFaturamento', 'DataEntrega']
    for coluna in colunas_para_validar:
        if coluna in dados.columns:
            dados[coluna] = dados[coluna].astype(str).replace('nan', '', regex=False)

    # Preenche qualquer valor NaN restante com string vazia
    dados.fillna('', inplace=True)

    # Exporta CSV limpo
    dados.to_csv(
        caminho_csv_teste,
        sep=',',
        index=False,
        encoding='utf-8',
        quoting=csv.QUOTE_ALL,
        lineterminator='\n'
    )

    print(f"✅ CSV limpo salvo com sucesso em: {caminho_csv_teste}")
else:
    print(f"⚠️ Arquivo não encontrado: {caminho_arquivo_origem}")

    import os
import pandas as pd
import unicodedata # Para limpar nomes de colunas
import re          # Para limpar nomes de colunas
import logging
from google.oauth2 import service_account # Para carregar credenciais do BigQuery
import pandas_gbq # Para enviar dados ao BigQuery

# --- Configuração do Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Caminhos e Nomes ---
caminho_pasta_dados = r'C:\Users\Florestas\Desktop\Relatorio Oficial 2025\Automação\Dados'
nome_arquivo_csv_limpo = 'consultapedidosvd.csv'
caminho_completo_csv_limpo = os.path.join(caminho_pasta_dados, nome_arquivo_csv_limpo)

# --- CONFIGURAÇÕES DO BIGQUERY ---
PROJECT_ID = 'grupo-florestas-429613'
TABLE_ID = 'VDHUB.consultapedidos'
ARQUIVO_CREDENCIAL_JSON = r'C:\Users\Florestas\Desktop\grupo-florestas-429613-080e980a456d.json'

# --- Nomes das Colunas para Conversão de Tipo Específica ---
COLUNAS_DATAS = [
    'datacaptacao', 'dataaprovacao', 'datafaturamento',
    'dataentrega', 'previsaoentrega'
    # Adicione outras colunas de data aqui, se necessário
]
COLUNAS_VALORES_FLOAT = ['valorliquido', 'valorpedido']

# --- Função para limpar nomes de colunas ---
def limpar_nome_coluna_bq(col_name):
    if not isinstance(col_name, str): col_name = str(col_name)
    nfkd_form = unicodedata.normalize('NFKD', col_name)
    ascii_name = nfkd_form.encode('ASCII', 'ignore').decode('ASCII')
    lower_name = ascii_name.lower()
    cleaned_name = re.sub(r'\s+', '_', lower_name)
    cleaned_name = re.sub(r'[^\w_]+', '', cleaned_name)
    if cleaned_name and cleaned_name[0].isdigit():
        cleaned_name = '_' + cleaned_name
    if not cleaned_name:
        cleaned_name = 'coluna_sem_nome'
    return cleaned_name

# --- Execução Principal ---
logging.info("--- INÍCIO DO SCRIPT DE ENVIO PARA BIGQUERY (consultapedidos) ---")
# ... (mensagens de log iniciais) ...

# 1. Autenticação (igual ao script anterior)
if not os.path.exists(ARQUIVO_CREDENCIAL_JSON):
    logging.error(f"!!! ERRO FATAL: Arquivo de Credenciais JSON não encontrado: '{ARQUIVO_CREDENCIAL_JSON}'.")
    exit()
try:
    credentials = service_account.Credentials.from_service_account_file(
        ARQUIVO_CREDENCIAL_JSON,
        scopes=["https://www.googleapis.com/auth/bigquery"]
    )
    logging.info(f"[OK] Credenciais carregadas: {ARQUIVO_CREDENCIAL_JSON}")
except Exception as auth_err:
    logging.error(f"!!! ERRO FATAL ao carregar credenciais: {auth_err}")
    exit()

# 2. Verifica se o arquivo CSV existe (igual ao script anterior)
if not os.path.exists(caminho_completo_csv_limpo):
    logging.error(f"!!! ERRO FATAL: Arquivo CSV de origem não encontrado: '{caminho_completo_csv_limpo}'.")
    exit()

# 3. Lê o CSV e Prepara o DataFrame
df_para_bq = None
try:
    logging.info(f"Lendo arquivo CSV: {nome_arquivo_csv_limpo}...")
    df_para_bq = pd.read_csv(caminho_completo_csv_limpo, dtype=str, keep_default_na=False, na_values=[''])

    if df_para_bq.empty:
        raise ValueError("O arquivo CSV está vazio.")
    logging.info(f"Leitura do CSV OK: {len(df_para_bq)} linhas.")

    logging.info("Limpando nomes das colunas para o BigQuery...")
    df_para_bq.columns = [limpar_nome_coluna_bq(col) for col in df_para_bq.columns]
    logging.info(f"Nomes das colunas limpos (ex: {list(df_para_bq.columns[:5])}...).")

    # Converte colunas de DATAS
    logging.info("Convertendo colunas de data para datetime no Pandas...")
    for col_data in COLUNAS_DATAS:
        if col_data in df_para_bq.columns:
            logging.info(f" -> Convertendo coluna de data: {col_data}")
            # ***** AJUSTE PRINCIPAL AQUI *****
            # Se suas datas no CSV estão como AAAA-MM-DD, use format='%Y-%m-%d'
            # Se estiverem como DD/MM/AAAA, use format='%d/%m/%Y' (e remova dayfirst=True ou mantenha-o)
            # Se estiverem como AAAAMMDD (sem separadores), use format='%Y%m%d'
            # Se o formato for variado, você pode remover o parâmetro 'format' e deixar o Pandas tentar inferir,
            # possivelmente com dayfirst=True se DD/MM/AAAA for comum:
            # df_para_bq[col_data] = pd.to_datetime(df_para_bq[col_data], errors='coerce', dayfirst=True)
            df_para_bq[col_data] = pd.to_datetime(df_para_bq[col_data], format='%Y-%m-%d', errors='coerce') # ASSUMINDO AAAA-MM-DD

            num_nulos = df_para_bq[col_data].isna().sum()
            if num_nulos > 0:
                logging.warning(f"     Coluna '{col_data}': {num_nulos} valores não puderam ser convertidos para data (formato esperado AAAA-MM-DD) e serão NULOS.")
        else:
            logging.warning(f"Coluna de data '{col_data}' não encontrada. Verifique a lista 'COLUNAS_DATAS'.")

    # Converte colunas de VALORES para FLOAT (igual ao script anterior)
    logging.info("Convertendo colunas de valor para numérico (float) no Pandas...")
    for col_valor in COLUNAS_VALORES_FLOAT:
        if col_valor in df_para_bq.columns:
            logging.info(f" -> Convertendo coluna de valor: {col_valor}")
            if df_para_bq[col_valor].dtype == 'object':
                df_para_bq[col_valor] = df_para_bq[col_valor].str.replace(r'[^\d.,-]', '', regex=True).str.replace(',', '.', regex=False)
            df_para_bq[col_valor] = pd.to_numeric(df_para_bq[col_valor], errors='coerce')
            num_nulos = df_para_bq[col_valor].isna().sum()
            if num_nulos > 0:
                logging.warning(f"     Coluna '{col_valor}': {num_nulos} valores não puderam ser convertidos para número (serão NULOS).")
        else:
            logging.warning(f"Coluna de valor '{col_valor}' não encontrada. Verifique a lista 'COLUNAS_VALORES_FLOAT'.")

    # Define o esquema da tabela para o BigQuery (igual ao script anterior)
    table_schema = []
    for col_name in df_para_bq.columns:
        if col_name in COLUNAS_DATAS:
            table_schema.append({'name': col_name, 'type': 'DATE'})
        elif col_name in COLUNAS_VALORES_FLOAT:
            table_schema.append({'name': col_name, 'type': 'FLOAT64'})
        else:
            table_schema.append({'name': col_name, 'type': 'STRING'})
    logging.info(f"Schema definido para o BigQuery: {table_schema}")

    logging.info("Preparação do DataFrame e schema para envio ao BigQuery concluída.")

except Exception as e:
    logging.error(f"!!! ERRO FATAL durante leitura ou preparação do CSV: {e}")
    import traceback
    traceback.print_exc()
    exit()

# 4. Envia o DataFrame para o BigQuery (igual ao script anterior)
if df_para_bq is not None and not df_para_bq.empty:
    logging.info(f"Enviando {len(df_para_bq)} linhas para BigQuery: '{PROJECT_ID}:{TABLE_ID}' (Substituindo)...")
    try:
        pandas_gbq.to_gbq(
            df_para_bq,
            destination_table=TABLE_ID,
            project_id=PROJECT_ID,
            credentials=credentials,
            if_exists='replace',
            table_schema=table_schema,
            progress_bar=True
        )
        logging.info(f"✅ SUCESSO! Dados enviados e tabela '{TABLE_ID}' foi substituída no BigQuery.")
    except Exception as e:
        logging.error(f"!!! ERRO AO ENVIAR PARA O BIGQUERY: {e}")
        import traceback
        traceback.print_exc()
else:
    logging.error("DataFrame está vazio ou não foi definido. Nenhum dado enviado.")

logging.info(f"--- FIM DO SCRIPT DE ENVIO PARA BIGQUERY ({TABLE_ID}) ---")


2025-06-07 14:10:38,137 - INFO - Iniciando processo de atualização de 'consultapedidos.xlsx'...
2025-06-07 14:10:38,150 - INFO - Arquivos encontrados em Downloads: 6
2025-06-07 14:10:38,151 - INFO -   -> ConsultaPedidos_5f2730d3-8095-4b0b-835e-3c77f208ae25.xlsx
2025-06-07 14:10:38,151 - INFO -   -> ConsultaPedidos_80c264b7-7880-4cfe-b64c-dbdb608ea4ac.xlsx
2025-06-07 14:10:38,152 - INFO -   -> ConsultaPedidos_815e7bf7-0373-412b-af30-10688b27f50a.xlsx
2025-06-07 14:10:38,152 - INFO -   -> ConsultaPedidos_b99d31ed-1019-4d60-a636-da2e90872919.xlsx
2025-06-07 14:10:38,154 - INFO -   -> ConsultaPedidos_d3ca485d-5fcc-4dc5-b8e8-d273d6b26a79.xlsx
2025-06-07 14:10:38,155 - INFO -   -> ConsultaPedidos_db7434da-5513-4582-b6f0-9fbf84306aa1.xlsx
2025-06-07 14:10:38,162 - INFO - Lendo arquivo: 'ConsultaPedidos_5f2730d3-8095-4b0b-835e-3c77f208ae25.xlsx'
2025-06-07 14:11:09,685 - INFO - Arquivo 'ConsultaPedidos_5f2730d3-8095-4b0b-835e-3c77f208ae25.xlsx' processado.
2025-06-07 14:11:09,685 - INFO - Lend

✅ CSV limpo salvo com sucesso em: C:\Users\Florestas\Desktop\Relatorio Oficial 2025\Automação\Dados\consultapedidosvd.csv


2025-06-07 14:16:07,507 - INFO - Leitura do CSV OK: 128558 linhas.
2025-06-07 14:16:07,507 - INFO - Limpando nomes das colunas para o BigQuery...
2025-06-07 14:16:07,507 - INFO - Nomes das colunas limpos (ex: ['codigopedido', 'situacaofiscal', 'notafiscal', 'qtdemateriais', 'qtdeitens']...).
2025-06-07 14:16:07,507 - INFO - Convertendo colunas de data para datetime no Pandas...
2025-06-07 14:16:07,507 - INFO -  -> Convertendo coluna de data: datacaptacao
2025-06-07 14:16:07,523 - INFO -  -> Convertendo coluna de data: dataaprovacao
2025-06-07 14:16:07,543 - INFO -  -> Convertendo coluna de data: datafaturamento
2025-06-07 14:16:07,556 - WARNING -      Coluna 'datafaturamento': 5 valores não puderam ser convertidos para data (formato esperado AAAA-MM-DD) e serão NULOS.
2025-06-07 14:16:07,557 - INFO -  -> Convertendo coluna de data: dataentrega
2025-06-07 14:16:07,569 - WARNING -      Coluna 'dataentrega': 17261 valores não puderam ser convertidos para data (formato esperado AAAA-MM-DD)

### Dados Transformados 
data, salvados para puxar para o bigquery, tivemos que deixar 

No painel de criação da tabela:

Formato: CSV

Delimitador: não precisa alterar (deixe como vírgula)

Esquema: pode manter “Detectar automaticamente”

Certifique-se de que a primeira linha do CSV contém os nomes das colunas

In [3]:
import os
import pandas as pd
import csv
import unicodedata
import re
from datetime import datetime

# Caminhos
caminho_destino = r'C:\Users\Florestas\Desktop\Relatorio Oficial 2025\Automação\Dados'
nome_padrao_destino = 'consultapedidos.xlsx'
caminho_arquivo_origem = os.path.join(caminho_destino, nome_padrao_destino)
caminho_csv_teste = os.path.join(caminho_destino, 'consultapedidosvd.csv')

# Função para limpar os nomes das colunas
def limpar_nome_coluna(col):
    if not isinstance(col, str):
        return col
    col = unicodedata.normalize('NFKD', col).encode('ASCII', 'ignore').decode('utf-8')
    col = re.sub(r'[^\w]', '', col)
    return col

# Função para converter datas para o formato yyyy-mm-dd
def converter_data_para_iso(valor):
    if isinstance(valor, str):
        # Tenta detectar datas no formato dd/mm/yyyy ou dd-mm-yyyy
        match = re.match(r'^(\d{2})[/-](\d{2})[/-](\d{4})$', valor.strip())
        if match:
            dia, mes, ano = match.groups()
            try:
                return f"{ano}-{mes}-{dia}"
            except:
                return valor
    return valor

if os.path.exists(caminho_arquivo_origem):
    dados = pd.read_excel(caminho_arquivo_origem, dtype=str)

    # Limpa os nomes das colunas
    dados.columns = [limpar_nome_coluna(col) for col in dados.columns]

    # Remove quebras de linha e espaços extras
    dados = dados.applymap(lambda x: str(x).replace('\n', ' ').strip() if pd.notna(x) else '')

    # Remove horas de colunas com datas
    for coluna in dados.columns:
        if any(keyword in coluna.lower() for keyword in ['data', 'dt']):
            # Remove horas do tipo "dd/mm/yyyy hh:mm"
            dados[coluna] = dados[coluna].apply(
                lambda x: re.sub(r'\s+\d{2}:\d{2}(:\d{2})?$', '', x) if isinstance(x, str) else x
            )
            # Converte para o formato yyyy-mm-dd
            dados[coluna] = dados[coluna].apply(converter_data_para_iso)

    # Substitui 'nan' por string vazia em colunas conhecidas
    colunas_para_validar = ['DataFaturamento', 'DataEntrega']
    for coluna in colunas_para_validar:
        if coluna in dados.columns:
            dados[coluna] = dados[coluna].astype(str).replace('nan', '', regex=False)

    # Preenche qualquer valor NaN restante com string vazia
    dados.fillna('', inplace=True)

    # Exporta CSV limpo
    dados.to_csv(
        caminho_csv_teste,
        sep=',',
        index=False,
        encoding='utf-8',
        quoting=csv.QUOTE_ALL,
        lineterminator='\n'
    )

    print(f"✅ CSV limpo salvo com sucesso em: {caminho_csv_teste}")
else:
    print(f"⚠️ Arquivo não encontrado: {caminho_arquivo_origem}")


C:\Users\Florestas\AppData\Local\Temp\ipykernel_15748\3834806206.py:42: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dados = dados.applymap(lambda x: str(x).replace('\n', ' ').strip() if pd.notna(x) else '')


✅ CSV limpo salvo com sucesso em: C:\Users\Florestas\Desktop\Relatorio Oficial 2025\Automação\Dados\consultapedidosvd.csv


### Enviar Bigquery

In [4]:
!pip install google-cloud-bigquery

In [5]:
!pip install pandas pandas_gbq

In [6]:
import os
import pandas as pd
import unicodedata # Para limpar nomes de colunas
import re          # Para limpar nomes de colunas
import logging
from google.oauth2 import service_account # Para carregar credenciais do BigQuery
import pandas_gbq # Para enviar dados ao BigQuery

# --- Configuração do Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Caminhos e Nomes ---
caminho_pasta_dados = r'C:\Users\Florestas\Desktop\Relatorio Oficial 2025\Automação\Dados'
nome_arquivo_csv_limpo = 'consultapedidosvd.csv'
caminho_completo_csv_limpo = os.path.join(caminho_pasta_dados, nome_arquivo_csv_limpo)

# --- CONFIGURAÇÕES DO BIGQUERY ---
PROJECT_ID = 'grupo-florestas-429613'
TABLE_ID = 'VDHUB.consultapedidos'
ARQUIVO_CREDENCIAL_JSON = r'C:\Users\Florestas\Desktop\grupo-florestas-429613-080e980a456d.json'

# --- Nomes das Colunas para Conversão de Tipo Específica ---
COLUNAS_DATAS = [
    'datacaptacao', 'dataaprovacao', 'datafaturamento',
    'dataentrega', 'previsaoentrega'
    # Adicione outras colunas de data aqui, se necessário
]
COLUNAS_VALORES_FLOAT = ['valorliquido', 'valorpedido']

# --- Função para limpar nomes de colunas ---
def limpar_nome_coluna_bq(col_name):
    if not isinstance(col_name, str): col_name = str(col_name)
    nfkd_form = unicodedata.normalize('NFKD', col_name)
    ascii_name = nfkd_form.encode('ASCII', 'ignore').decode('ASCII')
    lower_name = ascii_name.lower()
    cleaned_name = re.sub(r'\s+', '_', lower_name)
    cleaned_name = re.sub(r'[^\w_]+', '', cleaned_name)
    if cleaned_name and cleaned_name[0].isdigit():
        cleaned_name = '_' + cleaned_name
    if not cleaned_name:
        cleaned_name = 'coluna_sem_nome'
    return cleaned_name

# --- Execução Principal ---
logging.info("--- INÍCIO DO SCRIPT DE ENVIO PARA BIGQUERY (consultapedidos) ---")
# ... (mensagens de log iniciais) ...

# 1. Autenticação (igual ao script anterior)
if not os.path.exists(ARQUIVO_CREDENCIAL_JSON):
    logging.error(f"!!! ERRO FATAL: Arquivo de Credenciais JSON não encontrado: '{ARQUIVO_CREDENCIAL_JSON}'.")
    exit()
try:
    credentials = service_account.Credentials.from_service_account_file(
        ARQUIVO_CREDENCIAL_JSON,
        scopes=["https://www.googleapis.com/auth/bigquery"]
    )
    logging.info(f"[OK] Credenciais carregadas: {ARQUIVO_CREDENCIAL_JSON}")
except Exception as auth_err:
    logging.error(f"!!! ERRO FATAL ao carregar credenciais: {auth_err}")
    exit()

# 2. Verifica se o arquivo CSV existe (igual ao script anterior)
if not os.path.exists(caminho_completo_csv_limpo):
    logging.error(f"!!! ERRO FATAL: Arquivo CSV de origem não encontrado: '{caminho_completo_csv_limpo}'.")
    exit()

# 3. Lê o CSV e Prepara o DataFrame
df_para_bq = None
try:
    logging.info(f"Lendo arquivo CSV: {nome_arquivo_csv_limpo}...")
    df_para_bq = pd.read_csv(caminho_completo_csv_limpo, dtype=str, keep_default_na=False, na_values=[''])

    if df_para_bq.empty:
        raise ValueError("O arquivo CSV está vazio.")
    logging.info(f"Leitura do CSV OK: {len(df_para_bq)} linhas.")

    logging.info("Limpando nomes das colunas para o BigQuery...")
    df_para_bq.columns = [limpar_nome_coluna_bq(col) for col in df_para_bq.columns]
    logging.info(f"Nomes das colunas limpos (ex: {list(df_para_bq.columns[:5])}...).")

    # Converte colunas de DATAS
    logging.info("Convertendo colunas de data para datetime no Pandas...")
    for col_data in COLUNAS_DATAS:
        if col_data in df_para_bq.columns:
            logging.info(f" -> Convertendo coluna de data: {col_data}")
            # ***** AJUSTE PRINCIPAL AQUI *****
            # Se suas datas no CSV estão como AAAA-MM-DD, use format='%Y-%m-%d'
            # Se estiverem como DD/MM/AAAA, use format='%d/%m/%Y' (e remova dayfirst=True ou mantenha-o)
            # Se estiverem como AAAAMMDD (sem separadores), use format='%Y%m%d'
            # Se o formato for variado, você pode remover o parâmetro 'format' e deixar o Pandas tentar inferir,
            # possivelmente com dayfirst=True se DD/MM/AAAA for comum:
            # df_para_bq[col_data] = pd.to_datetime(df_para_bq[col_data], errors='coerce', dayfirst=True)
            df_para_bq[col_data] = pd.to_datetime(df_para_bq[col_data], format='%Y-%m-%d', errors='coerce') # ASSUMINDO AAAA-MM-DD

            num_nulos = df_para_bq[col_data].isna().sum()
            if num_nulos > 0:
                logging.warning(f"     Coluna '{col_data}': {num_nulos} valores não puderam ser convertidos para data (formato esperado AAAA-MM-DD) e serão NULOS.")
        else:
            logging.warning(f"Coluna de data '{col_data}' não encontrada. Verifique a lista 'COLUNAS_DATAS'.")

    # Converte colunas de VALORES para FLOAT (igual ao script anterior)
    logging.info("Convertendo colunas de valor para numérico (float) no Pandas...")
    for col_valor in COLUNAS_VALORES_FLOAT:
        if col_valor in df_para_bq.columns:
            logging.info(f" -> Convertendo coluna de valor: {col_valor}")
            if df_para_bq[col_valor].dtype == 'object':
                df_para_bq[col_valor] = df_para_bq[col_valor].str.replace(r'[^\d.,-]', '', regex=True).str.replace(',', '.', regex=False)
            df_para_bq[col_valor] = pd.to_numeric(df_para_bq[col_valor], errors='coerce')
            num_nulos = df_para_bq[col_valor].isna().sum()
            if num_nulos > 0:
                logging.warning(f"     Coluna '{col_valor}': {num_nulos} valores não puderam ser convertidos para número (serão NULOS).")
        else:
            logging.warning(f"Coluna de valor '{col_valor}' não encontrada. Verifique a lista 'COLUNAS_VALORES_FLOAT'.")

    # Define o esquema da tabela para o BigQuery (igual ao script anterior)
    table_schema = []
    for col_name in df_para_bq.columns:
        if col_name in COLUNAS_DATAS:
            table_schema.append({'name': col_name, 'type': 'DATE'})
        elif col_name in COLUNAS_VALORES_FLOAT:
            table_schema.append({'name': col_name, 'type': 'FLOAT64'})
        else:
            table_schema.append({'name': col_name, 'type': 'STRING'})
    logging.info(f"Schema definido para o BigQuery: {table_schema}")

    logging.info("Preparação do DataFrame e schema para envio ao BigQuery concluída.")

except Exception as e:
    logging.error(f"!!! ERRO FATAL durante leitura ou preparação do CSV: {e}")
    import traceback
    traceback.print_exc()
    exit()

# 4. Envia o DataFrame para o BigQuery (igual ao script anterior)
if df_para_bq is not None and not df_para_bq.empty:
    logging.info(f"Enviando {len(df_para_bq)} linhas para BigQuery: '{PROJECT_ID}:{TABLE_ID}' (Substituindo)...")
    try:
        pandas_gbq.to_gbq(
            df_para_bq,
            destination_table=TABLE_ID,
            project_id=PROJECT_ID,
            credentials=credentials,
            if_exists='replace',
            table_schema=table_schema,
            progress_bar=True
        )
        logging.info(f"✅ SUCESSO! Dados enviados e tabela '{TABLE_ID}' foi substituída no BigQuery.")
    except Exception as e:
        logging.error(f"!!! ERRO AO ENVIAR PARA O BIGQUERY: {e}")
        import traceback
        traceback.print_exc()
else:
    logging.error("DataFrame está vazio ou não foi definido. Nenhum dado enviado.")

logging.info(f"--- FIM DO SCRIPT DE ENVIO PARA BIGQUERY ({TABLE_ID}) ---")

2025-06-06 10:16:00,045 - INFO - --- INÍCIO DO SCRIPT DE ENVIO PARA BIGQUERY (consultapedidos) ---
2025-06-06 10:16:00,228 - INFO - [OK] Credenciais carregadas: C:\Users\Florestas\Desktop\grupo-florestas-429613-080e980a456d.json
2025-06-06 10:16:00,228 - INFO - Lendo arquivo CSV: consultapedidosvd.csv...
2025-06-06 10:16:01,836 - INFO - Leitura do CSV OK: 127464 linhas.
2025-06-06 10:16:01,843 - INFO - Limpando nomes das colunas para o BigQuery...
2025-06-06 10:16:01,843 - INFO - Nomes das colunas limpos (ex: ['codigopedido', 'situacaofiscal', 'notafiscal', 'qtdemateriais', 'qtdeitens']...).
2025-06-06 10:16:01,843 - INFO - Convertendo colunas de data para datetime no Pandas...
2025-06-06 10:16:01,851 - INFO -  -> Convertendo coluna de data: datacaptacao
2025-06-06 10:16:01,916 - INFO -  -> Convertendo coluna de data: dataaprovacao
2025-06-06 10:16:01,958 - INFO -  -> Convertendo coluna de data: datafaturamento
2025-06-06 10:16:01,991 - WARNING -      Coluna 'datafaturamento': 6 valore

## PL
em atualização, verificar se vai funcionar a automação

In [7]:
import os
import pandas as pd

# Caminhos
caminho_downloads = r'C:\Users\Florestas\Downloads'
caminho_destino = r'G:\Meu Drive\Relatorio Oficial 2025\Drive\Plataforma Logistica'
nome_arquivo_destino = 'PedidosPl.xlsx'

if not os.path.exists(caminho_destino):
    os.makedirs(caminho_destino)
    print(f"📁 Pasta criada: {caminho_destino}")

# Lista arquivos .csv
arquivos_csv = [
    os.path.join(caminho_downloads, f)
    for f in os.listdir(caminho_downloads)
    if f.lower().endswith('.csv')
]

if arquivos_csv:
    arquivo_mais_recente = max(arquivos_csv, key=os.path.getmtime)
    print(f"📥 Arquivo mais recente encontrado: {arquivo_mais_recente}")

    # Tentativa de leitura com auto-separador e encoding
    try:
        dados = pd.read_csv(
            arquivo_mais_recente,
            sep=None,
            engine='python',
            encoding='utf-8',
            dtype=str
        )
    except UnicodeDecodeError:
        dados = pd.read_csv(
            arquivo_mais_recente,
            sep=None,
            engine='python',
            encoding='latin1',
            dtype=str
        )

    # Remove linhas que estiverem com mais colunas ou valores bagunçados
    num_colunas = len(dados.columns)
    dados = dados[dados.apply(lambda row: row.count() > num_colunas // 2, axis=1)]  # mantém linhas com pelo menos metade dos dados

    caminho_arquivo_destino = os.path.join(caminho_destino, nome_arquivo_destino)

    if os.path.exists(caminho_arquivo_destino):
        dados_existentes = pd.read_excel(caminho_arquivo_destino, dtype=str)

        # Concatena, remove duplicados com base na coluna "Pedido" (se existir)
        if 'Pedido' in dados.columns and 'Pedido' in dados_existentes.columns:
            dados_combinados = pd.concat([dados_existentes, dados], ignore_index=True)
            dados_combinados = dados_combinados.drop_duplicates(subset='Pedido', keep='last')
        else:
            dados_combinados = pd.concat([dados_existentes, dados], ignore_index=True)

        dados_combinados.to_excel(caminho_arquivo_destino, index=False)
        print(f"✅ Arquivo atualizado: {nome_arquivo_destino}")
    else:
        dados.to_excel(caminho_arquivo_destino, index=False)
        print(f"✅ Arquivo criado: {nome_arquivo_destino}")

    # # Remove o CSV original
    # os.remove(arquivo_mais_recente)
    # print(f"🗑️ Arquivo original removido: {arquivo_mais_recente}")
else:
    print("⚠️ Nenhum arquivo .csv encontrado na pasta de downloads.")


📥 Arquivo mais recente encontrado: C:\Users\Florestas\Downloads\3633d422a83fa307eb3e31fb07b40d96ebf1cb8d.csv
✅ Arquivo atualizado: PedidosPl.xlsx


### Dados Transformados para enviar para o banco de dados

In [8]:
import pandas as pd
import os
import logging
import csv

# --- Configurações de Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Caminhos ---
caminho_pasta = r'G:\Meu Drive\Relatorio Oficial 2025\Drive\Plataforma Logistica'
nome_arquivo_excel_origem = 'PedidosPl.xlsx'
nome_arquivo_csv_destino = 'PedidosPl_para_BQ.csv'
caminho_completo_excel_origem = os.path.join(caminho_pasta, nome_arquivo_excel_origem)
caminho_completo_csv_destino = os.path.join(caminho_pasta, nome_arquivo_csv_destino)

# --- Mapeamento de colunas: Excel -> BigQuery ---
colunas_mapeadas = {
    'Pedido': 'pedido',
    'Data de criação': 'data_criacao',
    'Data de Coleta': 'data_coleta',
    'Nota 1 Data': 'nota1_data',
    'Data aprovação': 'data_aprovacao',
    'Data de contratação': 'data_contratacao',
    'Prazo Cliente': 'prazo_cliente',
    'Prazo': 'prazo',
    'Valor do frete': 'valor_frete',
    'Preço do frete': 'preco_frete',
    'Valor Total': 'valor_total',
    'UF': 'uf',
    'Município': 'municipio',
    'CEP': 'cep',
    'Bairro': 'bairro',
    'Endereço': 'endereco',
    'Número': 'numero',
    'Complemento': 'complemento',
    'CPF/CNPJ': 'cpf_cnpj',
    'Tipo de Pessoa': 'tipo_pessoa',
    'Nome': 'nome',
    'Rota': 'rota',
    'Itens': 'itens',
    'Peso (Kg)': 'peso_kg',
    'Motorista': 'motorista',
    'Transportadora': 'transportadora',
    'Status': 'status',
    'Status (hora efetuada)': 'status_hora_efetuada',
    'Status (detalhe)': 'status_detalhe'
}

coluna_chave = 'Pedido'

# Define colunas de data e datetime
colunas_datas_simples = [
    'Data de criação', 'Data de Coleta', 'Nota 1 Data',
    'Data aprovação', 'Data de contratação'
]
colunas_datetime = ['Status (hora efetuada)']

def format_date_columns(df, colunas_data, colunas_datahora):
    df_copy = df.copy()

    for col in colunas_data:
        if col in df_copy.columns:
            try:
                convertidas = pd.to_datetime(df_copy[col], errors='coerce', dayfirst=True)
                df_copy[col] = convertidas.dt.strftime('%Y-%m-%d')
                df_copy[col].fillna('', inplace=True)
                if convertidas.isna().sum() > 0:
                    logging.warning(f"Coluna '{col}': {convertidas.isna().sum()} datas não puderam ser convertidas.")
            except Exception as e:
                logging.warning(f"Erro ao converter coluna '{col}': {e}")

    for col in colunas_datahora:
        if col in df_copy.columns:
            try:
                df_copy[col] = df_copy[col].astype(str).str.split("T").str[0]
                df_copy[col] = pd.to_datetime(df_copy[col], errors='coerce').dt.strftime('%Y-%m-%d')
                df_copy[col].fillna('', inplace=True)
            except Exception as e:
                logging.warning(f"Erro ao dividir/formatar coluna '{col}': {e}")
    
    return df_copy

# --- Execução Principal ---
logging.info("-----------------------------------------------------")
logging.info(f"Iniciando processamento de '{nome_arquivo_excel_origem}'")
logging.info(f"Gerando CSV '{nome_arquivo_csv_destino}' para BigQuery")
logging.info("-----------------------------------------------------")

if not os.path.exists(caminho_completo_excel_origem):
    logging.error(f"Arquivo não encontrado: '{caminho_completo_excel_origem}'")
    exit()

try:
    df_source = pd.read_excel(caminho_completo_excel_origem, dtype=str)
    if df_source.empty:
        logging.error("Arquivo Excel está vazio.")
        exit()
    logging.info(f"Leitura concluída com {len(df_source)} linhas.")

    colunas_existentes = [col for col in colunas_mapeadas.keys() if col in df_source.columns]
    colunas_faltando = [col for col in colunas_mapeadas.keys() if col not in df_source.columns]

    if not colunas_existentes:
        logging.error("Nenhuma das colunas especificadas foi encontrada.")
        exit()
    if colunas_faltando:
        logging.warning(f"Colunas não encontradas no Excel: {colunas_faltando}")
    if coluna_chave not in colunas_existentes:
        logging.error(f"Coluna chave '{coluna_chave}' não encontrada.")
        exit()

    # Filtra colunas e renomeia
    df = df_source[colunas_existentes].copy()
    df = format_date_columns(df, colunas_datas_simples, colunas_datetime)
    df.rename(columns={k: v for k, v in colunas_mapeadas.items() if k in df.columns}, inplace=True)

    # Limpeza
    df.dropna(how='all', inplace=True)
    for col in df.select_dtypes(include='object'):
        df[col] = df[col].str.strip()
    df.dropna(subset=[colunas_mapeadas[coluna_chave]], inplace=True)
    df = df[df[colunas_mapeadas[coluna_chave]].astype(str).str.strip() != '']
    logging.info(f"{len(df)} linhas após limpeza.")

    # Remove duplicatas
    df.drop_duplicates(subset=[colunas_mapeadas[coluna_chave]], keep='last', inplace=True)

    # Exporta CSV
    df.fillna('', inplace=True)
    df.to_csv(caminho_completo_csv_destino, sep=',', encoding='utf-8-sig', index=False,
              quotechar='"', quoting=csv.QUOTE_MINIMAL)
    logging.info(f"✅ CSV salvo com sucesso em: '{caminho_completo_csv_destino}'")

except Exception as e:
    logging.error(f"Erro durante o processamento: {e}")
    import traceback
    traceback.print_exc()

logging.info("Processamento concluído.")


2025-06-06 10:26:58,393 - INFO - -----------------------------------------------------
2025-06-06 10:26:58,399 - INFO - Iniciando processamento de 'PedidosPl.xlsx'
2025-06-06 10:26:58,399 - INFO - Gerando CSV 'PedidosPl_para_BQ.csv' para BigQuery
2025-06-06 10:26:58,405 - INFO - -----------------------------------------------------


2025-06-06 10:32:23,776 - INFO - Leitura concluída com 86117 linhas.
C:\Users\Florestas\AppData\Local\Temp\ipykernel_15748\1280868867.py:64: UserWarning: Parsing dates in %Y-%m-%dT%H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  convertidas = pd.to_datetime(df_copy[col], errors='coerce', dayfirst=True)
C:\Users\Florestas\AppData\Local\Temp\ipykernel_15748\1280868867.py:66: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_copy[col].fillna('', inplace=True)
20

In [9]:
import os
import pandas as pd
import unicodedata # Para a função limpar_nome_coluna_bq
import re          # Para a função limpar_nome_coluna_bq
import logging
from google.oauth2 import service_account # Para carregar as credenciais do BigQuery
import pandas_gbq # Para usar pandas_gbq.to_gbq()

# --- Configuração do Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Caminhos e Nomes (AJUSTE CONFORME SEU AMBIENTE) ---
# Caminho onde o CSV 'PedidosPl_para_BQ.csv' foi salvo pelo seu script anterior
caminho_pasta_dados = r'G:\Meu Drive\Relatorio Oficial 2025\Drive\Plataforma Logistica'
nome_arquivo_csv_processado = 'PedidosPl_para_BQ.csv'
caminho_completo_csv_processado = os.path.join(caminho_pasta_dados, nome_arquivo_csv_processado)

# --- CONFIGURAÇÕES DO BIGQUERY (AJUSTE SE NECESSÁRIO) ---
PROJECT_ID = 'grupo-florestas-429613' # SEU ID do Projeto Google Cloud
DATASET_ID = 'VDHUB'
TABLE_NAME = 'pedidos_pl'
DESTINATION_TABLE = f'{DATASET_ID}.{TABLE_NAME}' # Tabela a ser SUBSTITUÍDA

# Caminho para o seu arquivo JSON de credenciais do Google Cloud
ARQUIVO_CREDENCIAL_JSON = r'C:\Users\Florestas\Desktop\grupo-florestas-429613-080e980a456d.json' # CONFIRME ESTE CAMINHO

# --- Colunas para Conversão de Tipo ---
# Nomes das colunas como estarão no CSV (após o mapeamento do seu script de preparação)
# Seu script de preparação já formata estas para YYYY-MM-DD
colunas_para_converter_data = [
    'data_criacao', 'data_coleta', 'nota1_data',
    'data_aprovacao', 'data_contratacao', 'prazo_cliente', 'prazo',
    'status_hora_efetuada' # Seu script anterior formata esta para YYYY-MM-DD também
]

# Colunas que devem ser numéricas no BigQuery
colunas_para_converter_numerico = [
    'valor_frete', 'preco_frete', 'valor_total', 'peso_kg'
    # Adicione outras colunas se forem numéricas, ex: 'numero' se for sempre numérico e não um identificador
]


# --- Funções Auxiliares ---
def limpar_nome_coluna_bq(col_name):
    """Limpa o nome da coluna para conformidade com o BigQuery."""
    if not isinstance(col_name, str): col_name = str(col_name)
    nfkd_form = unicodedata.normalize('NFKD', col_name)
    ascii_name = nfkd_form.encode('ASCII', 'ignore').decode('ASCII')
    lower_name = ascii_name.lower()
    cleaned_name = re.sub(r'\s+', '_', lower_name)
    cleaned_name = re.sub(r'[^\w_]+', '', cleaned_name)
    if cleaned_name and cleaned_name[0].isdigit():
        cleaned_name = '_' + cleaned_name
    if not cleaned_name:
        cleaned_name = 'coluna_sem_nome'
    return cleaned_name

# --- Execução Principal ---
logging.info("-----------------------------------------------------")
logging.info(f"Iniciando envio do arquivo CSV '{nome_arquivo_csv_processado}' para BigQuery.")
logging.info(f"Origem do CSV: {caminho_completo_csv_processado}")
logging.info(f"Destino no BigQuery: {PROJECT_ID}:{DESTINATION_TABLE}")
logging.info("A tabela no BigQuery será SUBSTITUÍDA.")
logging.info("-----------------------------------------------------")

# 1. Autenticar (Verifica arquivo de credenciais)
if not os.path.exists(ARQUIVO_CREDENCIAL_JSON):
    logging.error(f"!!! ERRO FATAL: Arquivo de Credenciais JSON não encontrado: '{ARQUIVO_CREDENCIAL_JSON}'. Verifique o caminho.")
    exit()
try:
    credentials = service_account.Credentials.from_service_account_file(
        ARQUIVO_CREDENCIAL_JSON,
        scopes=["https://www.googleapis.com/auth/bigquery"]
    )
    logging.info(f"[OK] Credenciais carregadas do arquivo JSON: {ARQUIVO_CREDENCIAL_JSON}")
except Exception as auth_err:
    logging.error(f"!!! ERRO FATAL: Falha ao carregar credenciais do arquivo JSON: {auth_err}")
    exit()

# 2. Verifica se o arquivo CSV processado de origem existe
if not os.path.exists(caminho_completo_csv_processado):
    logging.error(f"!!! ERRO FATAL: Arquivo CSV processado de origem não encontrado: '{caminho_completo_csv_processado}'.")
    logging.error("Certifique-se de que o script de preparação (que gera este CSV) foi executado com sucesso.")
    exit()

# 3. Lê o CSV Processado e Prepara o DataFrame
df_para_bq = None
try:
    logging.info(f"Lendo arquivo CSV processado: {nome_arquivo_csv_processado}...")
    # Seu script de preparação já preenche NaNs com '' e formata datas como string.
    # Ler tudo como string inicialmente para controle, depois converter tipos.
    df_para_bq = pd.read_csv(caminho_completo_csv_processado, dtype=str, keep_default_na=False, na_values=[''])

    if df_para_bq.empty:
        raise ValueError("O arquivo CSV processado está vazio ou não pôde ser lido corretamente.")
    logging.info(f"Leitura do CSV processado OK: {len(df_para_bq)} linhas.")

    # Limpa nomes das colunas para conformidade com BigQuery (seu script já faz um bom trabalho, mas isso é uma garantia)
    logging.info("Limpando nomes das colunas para o BigQuery...")
    df_para_bq.columns = [limpar_nome_coluna_bq(col) for col in df_para_bq.columns]
    # Garante que os nomes das colunas nas listas de conversão também estejam no formato limpo
    colunas_para_converter_data = [limpar_nome_coluna_bq(col) for col in colunas_para_converter_data]
    colunas_para_converter_numerico = [limpar_nome_coluna_bq(col) for col in colunas_para_converter_numerico]
    logging.info(f"Nomes das colunas limpos (ex: {list(df_para_bq.columns[:5])}...).")


    # Converte colunas de data (que devem estar como string 'YYYY-MM-DD' ou vazias)
    logging.info("Convertendo colunas de data para o tipo datetime (esperando formato YYYY-MM-DD)...")
    for coluna_data in colunas_para_converter_data:
        if coluna_data in df_para_bq.columns:
            logging.info(f" -> Convertendo coluna de data: {coluna_data}")
            # errors='coerce' transformará strings vazias ou formatos inválidos em NaT (Not a Time).
            df_para_bq[coluna_data] = pd.to_datetime(df_para_bq[coluna_data], errors='coerce', format='%Y-%m-%d')
            # Log de nulos (opcional, mas útil para depuração)
            num_nulos_dt = df_para_bq[coluna_data].isna().sum()
            if num_nulos_dt > 0:
                logging.info(f"      Coluna '{coluna_data}': {num_nulos_dt} valores são nulos (NaT) após conversão.")
        else:
            logging.warning(f"Coluna de data '{coluna_data}' não encontrada no DataFrame após limpeza dos nomes. Verifique a lista.")

    # Converte colunas numéricas
    logging.info(f"Convertendo colunas numéricas para o tipo float/numeric...")
    for coluna_num in colunas_para_converter_numerico:
        if coluna_num in df_para_bq.columns:
            logging.info(f" -> Convertendo coluna numérica: {coluna_num}")
            # Tratar possíveis vírgulas como separador decimal antes de converter para numérico
            # Seu script de limpeza não parece tratar isso, então é bom garantir aqui.
            if df_para_bq[coluna_num].dtype == 'object': # Se for string
                 df_para_bq[coluna_num] = df_para_bq[coluna_num].str.replace(',', '.', regex=False)
            # errors='coerce' transformará strings vazias ou não numéricas em NaN.
            df_para_bq[coluna_num] = pd.to_numeric(df_para_bq[coluna_num], errors='coerce')
            # Log de nulos (opcional)
            num_nulos_num = df_para_bq[coluna_num].isna().sum()
            if num_nulos_num > 0:
                logging.info(f"      Coluna '{coluna_num}': {num_nulos_num} valores são nulos (NaN) após conversão.")
        else:
            logging.warning(f"Coluna numérica '{coluna_num}' não encontrada no DataFrame após limpeza. Verifique a lista.")

    logging.info("Preparação do DataFrame para envio ao BigQuery concluída.")

except Exception as e:
    logging.error(f"!!! ERRO FATAL durante leitura ou preparação do CSV processado: {e}")
    import traceback
    traceback.print_exc()
    exit()

# 4. Envia o DataFrame Final para o BigQuery
if df_para_bq is not None and not df_para_bq.empty:
    try:
        logging.info(f"Enviando {len(df_para_bq)} linhas para BigQuery: '{PROJECT_ID}:{DESTINATION_TABLE}' (Substituindo tabela)...")
        
        pandas_gbq.to_gbq(
            df_para_bq,
            destination_table=DESTINATION_TABLE,
            project_id=PROJECT_ID,
            credentials=credentials,
            if_exists='replace', # SUBSTITUI A TABELA!
            progress_bar=True
        )

        logging.info(f"✅ SUCESSO! Dados enviados e tabela '{DESTINATION_TABLE}' foi substituída no BigQuery.")

    except Exception as e:
        logging.error(f"!!! ERRO AO ENVIAR PARA O BIGQUERY: {e}")
        logging.error("Verifique: PROJECT_ID, DESTINATION_TABLE, API do BigQuery Ativa, Permissões da Conta de Serviço (precisa de 'Editor de Dados do BigQuery' e 'Usuário de Tarefas do BigQuery'), Tipos de Dados no DataFrame.")
        import traceback
        traceback.print_exc()
else:
    logging.error("DataFrame 'df_para_bq' está vazio ou não foi definido. Nenhum dado enviado para o BigQuery.")

logging.info(f"--- FIM DO SCRIPT DE ENVIO PARA BIGQUERY ({DESTINATION_TABLE}) ---")

2025-06-06 10:32:33,299 - INFO - -----------------------------------------------------
2025-06-06 10:32:33,299 - INFO - Iniciando envio do arquivo CSV 'PedidosPl_para_BQ.csv' para BigQuery.
2025-06-06 10:32:33,306 - INFO - Origem do CSV: G:\Meu Drive\Relatorio Oficial 2025\Drive\Plataforma Logistica\PedidosPl_para_BQ.csv
2025-06-06 10:32:33,309 - INFO - Destino no BigQuery: grupo-florestas-429613:VDHUB.pedidos_pl
2025-06-06 10:32:33,311 - INFO - A tabela no BigQuery será SUBSTITUÍDA.
2025-06-06 10:32:33,314 - INFO - -----------------------------------------------------


2025-06-06 10:32:33,445 - INFO - [OK] Credenciais carregadas do arquivo JSON: C:\Users\Florestas\Desktop\grupo-florestas-429613-080e980a456d.json
2025-06-06 10:32:33,512 - INFO - Lendo arquivo CSV processado: PedidosPl_para_BQ.csv...
2025-06-06 10:32:34,992 - INFO - Leitura do CSV processado OK: 86116 linhas.
2025-06-06 10:32:34,994 - INFO - Limpando nomes das colunas para o BigQuery...
2025-06-06 10:32:34,999 - INFO - Nomes das colunas limpos (ex: ['pedido', 'data_criacao', 'data_coleta', 'nota1_data', 'data_aprovacao']...).
2025-06-06 10:32:34,999 - INFO - Convertendo colunas de data para o tipo datetime (esperando formato YYYY-MM-DD)...
2025-06-06 10:32:35,002 - INFO -  -> Convertendo coluna de data: data_criacao
2025-06-06 10:32:35,039 - INFO -  -> Convertendo coluna de data: data_coleta
2025-06-06 10:32:35,069 - INFO -       Coluna 'data_coleta': 996 valores são nulos (NaT) após conversão.
2025-06-06 10:32:35,069 - INFO -  -> Convertendo coluna de data: nota1_data
2025-06-06 10:32

# Transportadora 

médias de caixas e km, para enviar para bigquery

In [ ]:
# Script ATUALIZADO para ler Excel, SELECIONAR COLUNAS, limpar N/A,
# DIVIDIR Quantidade, PROCESSAR Distancia, formatar datas, remover duplicatas,
# limpar nomes colunas e salvar como Transportadora.csv.

import pandas as pd
import os
import logging
import csv
import re
import unicodedata

# --- Configurações ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Caminhos
caminho_pasta_origem_excel = r'C:\Users\Florestas\Desktop\Relatorio Oficial 2025\Excel - Pedidos'
nome_arquivo_excel_origem = 'Transportadora (Qtd de caixas, Km).xlsx'
caminho_completo_excel_origem = os.path.join(caminho_pasta_origem_excel, nome_arquivo_excel_origem)

caminho_pasta_destino_csv = r'G:\Meu Drive\Relatorio Oficial 2025\Drive\Custos Transportadora'
nome_arquivo_csv_destino = 'Transportadora.csv'
caminho_completo_csv_destino = os.path.join(caminho_pasta_destino_csv, nome_arquivo_csv_destino)

# --- Definições de Colunas ---

# ====> Colunas que você quer MANTER/PROCESSAR do Excel <====
# !!! VERIFIQUE OS NOMES EXATOS NO SEU EXCEL !!!
# Inclua a coluna original que será dividida (ex: 'Quantidade de Pacotes')
colunas_selecionadas = [
    'Número da Rota',        # Coluna Chave
    'Status',
    'Quantidade de Pacotes', # <<< COLUNA A SER DIVIDIDA (VERIFIQUE NOME!)
    'Data de Atualização',   # <<< Coluna de Data
    'Distância',             # <<< Coluna a ser processada (VERIFIQUE NOME!)
    'Transportadora'
]

# Coluna a ser dividida e os nomes das novas colunas resultantes
coluna_para_dividir = 'Quantidade de Pacotes' # <<< NOME EXATO NO EXCEL
nova_coluna_prevista = 'Qtd Prevista'         # Nome da 1ª nova coluna
nova_coluna_realizada = 'Qtd Realizada'       # Nome da 2ª nova coluna

# Coluna de distância a ser limpa
coluna_distancia = 'Distância' # <<< NOME EXATO NO EXCEL

# Coluna chave para identificar rotas únicas (DEVE estar em colunas_selecionadas)
coluna_chave = 'Número da Rota'

# Colunas de DATA (que estão em colunas_selecionadas) para formatar AAAA-MM-DD
colunas_datas_simples = [ 'Data de Atualização' ]

# --- Funções Auxiliares ---
def format_date_columns(df, colunas_data):
    # (Mesma função das respostas anteriores)
    if df is None or df.empty: return df
    df_copy = df.copy()
    logging.info(f"Formatando datas para AAAA-MM-DD nas colunas: {colunas_data}...")
    for col in colunas_data:
        if col in df_copy.columns:
            coluna_original_na_count = df_copy[col].isna().sum()
            coluna_str = df_copy[col].astype(str)
            try:
                converted_dates = pd.to_datetime(coluna_str, errors='coerce', dayfirst=True, infer_datetime_format=True)
                falhou_inicial = converted_dates.isna()
                if falhou_inicial.any():
                    try:
                        converted_dates.loc[falhou_inicial] = pd.to_datetime(coluna_str[falhou_inicial], errors='coerce', format='%Y-%m-%d')
                    except ValueError: pass
                df_copy[col] = converted_dates.dt.strftime('%Y-%m-%d')
                df_copy[col].fillna('', inplace=True)
                num_falhas_final = converted_dates.isna().sum()
                novas_falhas = num_falhas_final - coluna_original_na_count
                if novas_falhas > 0: logging.warning(f"Coluna '{col}': {novas_falhas} valores não formatados (ficarão vazios).")
            except Exception as e:
                logging.error(f"Erro ao converter coluna de data '{col}': {e}")
                df_copy[col] = ''
        else:
             logging.warning(f"Coluna de data '{col}' definida para formatação não encontrada.")
    return df_copy

def limpar_nome_coluna(col_name):
    # (Mesma função das respostas anteriores)
    if not isinstance(col_name, str): col_name = str(col_name)
    nfkd_form = unicodedata.normalize('NFKD', col_name)
    ascii_name = nfkd_form.encode('ASCII', 'ignore').decode('ASCII')
    lower_name = ascii_name.lower()
    cleaned_name = re.sub(r'\s+', '_', lower_name)
    cleaned_name = re.sub(r'[^\w_]+', '', cleaned_name)
    cleaned_name = re.sub(r'_+', '_', cleaned_name)
    cleaned_name = cleaned_name.strip('_')
    if cleaned_name and cleaned_name[0].isdigit(): cleaned_name = '_' + cleaned_name
    if not cleaned_name: cleaned_name = 'coluna_vazia'
    return cleaned_name

# --- Execução Principal ---
logging.info("-----------------------------------------------------")
logging.info(f"Processando '{nome_arquivo_excel_origem}' para gerar '{nome_arquivo_csv_destino}'.")
logging.info("Selecionando colunas, limpando N/A, DIVIDINDO Quantidade, LIMPANDO Distância, formatando datas, etc.")
logging.info("-----------------------------------------------------")

# 1. Garante que a pasta de destino do CSV exista
try:
    os.makedirs(caminho_pasta_destino_csv, exist_ok=True)
    logging.info(f"[OK] Pasta de destino CSV verificada/criada: '{caminho_pasta_destino_csv}'")
except Exception as e:
    logging.error(f"!!! ERRO FATAL: Falha ao verificar/criar pasta de destino: {e}")
    exit()

# 2. Verifica se o arquivo Excel de origem existe
if not os.path.exists(caminho_completo_excel_origem):
    logging.error(f"!!! ERRO FATAL: Arquivo Excel de ORIGEM não encontrado: '{caminho_completo_excel_origem}'.")
    exit()

# 3. Lê o arquivo Excel de origem
df_source = None
try:
    logging.info(f"Lendo arquivo Excel: {nome_arquivo_excel_origem}...")
    df_source = pd.read_excel(caminho_completo_excel_origem, dtype=str)
    if df_source is None or df_source.empty: raise ValueError("Planilha Excel vazia.")
    logging.info(f"Leitura inicial completa: {len(df_source)} linhas.")

    # 4. Filtra APENAS as colunas desejadas (incluindo a que será dividida)
    logging.info(f"Selecionando colunas: {colunas_selecionadas}...")
    colunas_encontradas = [col for col in colunas_selecionadas if col in df_source.columns]
    colunas_faltantes = [col for col in colunas_selecionadas if col not in df_source.columns]
    if not colunas_encontradas: raise ValueError("Nenhuma coluna desejada foi encontrada.")
    if colunas_faltantes: logging.warning(f"Colunas NÃO encontradas: {colunas_faltantes}")
    if coluna_chave not in colunas_encontradas: raise ValueError(f"Coluna chave '{coluna_chave}' não encontrada/selecionada.")
    if coluna_para_dividir not in colunas_encontradas: logging.warning(f"Coluna '{coluna_para_dividir}' para divisão não encontrada/selecionada.")
    if coluna_distancia not in colunas_encontradas: logging.warning(f"Coluna '{coluna_distancia}' para limpeza não encontrada/selecionada.")

    df_processed = df_source[colunas_encontradas].copy()
    logging.info(f"Colunas mantidas inicialmente: {list(df_processed.columns)}")

    # 5. Limpeza de Dados (N/A, Espaços, Chave Vazia)
    logging.info("Realizando limpeza básica (espaços, linhas vazias, N/A)...")
    df_processed.dropna(how='all', inplace=True)
    for col in df_processed.columns:
        if df_processed[col].dtype == 'object': df_processed[col] = df_processed[col].str.strip()
    logging.info("Substituindo 'N/A' por vazio...")
    df_processed.replace('N/A', '', inplace=True)
    df_processed.dropna(subset=[coluna_chave], inplace=True)
    df_processed = df_processed[df_processed[coluna_chave].astype(str).str.strip() != '']
    if df_processed.empty: raise ValueError(f"Nenhuma linha válida com '{coluna_chave}' após limpeza.")
    logging.info(f"Linhas após limpeza: {len(df_processed)}")

    # ****** ETAPA DE PROCESSAMENTO ESPECÍFICO ******
    # 5a. Dividir Coluna de Quantidade (ex: "31 / 31" -> colunas 'Qtd Prevista', 'Qtd Realizada')
    if coluna_para_dividir in df_processed.columns:
        logging.info(f"Dividindo coluna '{coluna_para_dividir}'...")
        # Divide pela barra '/', cria novas colunas temporárias
        split_cols = df_processed[coluna_para_dividir].astype(str).str.split('/', n=1, expand=True)
        # Atribui às novas colunas no dataframe principal, limpando espaços
        df_processed[nova_coluna_prevista] = split_cols[0].str.strip()
        df_processed[nova_coluna_realizada] = split_cols[1].str.strip() if split_cols.shape[1] > 1 else ''
        # Tenta converter para número
        df_processed[nova_coluna_prevista] = pd.to_numeric(df_processed[nova_coluna_prevista], errors='coerce')
        df_processed[nova_coluna_realizada] = pd.to_numeric(df_processed[nova_coluna_realizada], errors='coerce')
        # Remove a coluna original
        df_processed.drop(columns=[coluna_para_dividir], inplace=True)
        logging.info(f" -> Colunas '{nova_coluna_prevista}' e '{nova_coluna_realizada}' criadas.")
    else:
        logging.warning(f"Coluna '{coluna_para_dividir}' não encontrada para divisão.")

    # 5b. Limpar Coluna de Distância (ex: "82,9 Km" -> 82.9)
    if coluna_distancia in df_processed.columns:
        logging.info(f"Limpando e convertendo coluna '{coluna_distancia}'...")
        # Remove "km" (case-insensitive) e espaços, troca vírgula por ponto, converte para número
        df_processed[coluna_distancia] = pd.to_numeric(
            df_processed[coluna_distancia].astype(str).str.replace(r'(?i)\s*km', '', regex=True).str.replace(',', '.', regex=False).str.strip(),
            errors='coerce'
        )
        dist_nulos = df_processed[coluna_distancia].isna().sum()
        if dist_nulos > 0: logging.warning(f" -> {dist_nulos} valores em '{coluna_distancia}' não convertidos para número.")
        logging.info(f" -> Coluna '{coluna_distancia}' processada.")
    else:
        logging.warning(f"Coluna '{coluna_distancia}' não encontrada para limpeza.")
    # ***************************************************

    # 6. Remove Duplicatas baseado na chave
    logging.info(f"Removendo rotas duplicadas baseado em '{coluna_chave}' (mantendo a última)...")
    df_processed.drop_duplicates(subset=[coluna_chave], keep='last', inplace=True)
    logging.info(f"Linhas após remoção de duplicatas: {len(df_processed)}")

    # 7. Formata as Colunas de Data
    logging.info("Formatando colunas de data...")
    df_processed = format_date_columns(df_processed, colunas_datas_simples)

    # 8. Limpa Nomes das Colunas ANTES de Salvar
    logging.info("Limpando nomes das colunas para o CSV final...")
    df_processed.columns = [limpar_nome_coluna(col) for col in df_processed.columns]
    # Pega o nome limpo da coluna chave para referência (opcional)
    chave_limpa = limpar_nome_coluna(coluna_chave)
    logging.info(f"Nomes das colunas limpos (ex: '{coluna_chave}' virou '{chave_limpa}').")

    # 9. Salva o Resultado Final como CSV
    if not df_processed.empty:
        logging.info(f"Salvando dados processados em CSV: '{caminho_completo_csv_destino}'...")
        df_processed.fillna('', inplace=True) # Garante que nulos/NaN virem strings vazias
        df_processed.to_csv(caminho_completo_csv_destino, sep=',', encoding='utf-8-sig', index=False, quotechar='"', quoting=csv.QUOTE_MINIMAL)
        logging.info(f"✅ Arquivo CSV '{nome_arquivo_csv_destino}' criado/atualizado com sucesso em '{caminho_pasta_destino_csv}'.")
    else:
         logging.warning("DataFrame final está vazio após processamento. Nenhum dado foi salvo.")

except FileNotFoundError:
    logging.error(f"!!! ERRO FATAL: Arquivo Excel não encontrado em '{caminho_completo_excel_origem}'.")
except ValueError as ve:
    logging.error(f"!!! ERRO FATAL: Problema com os dados ou colunas: {ve}")
except PermissionError:
    logging.error(f"!!! ERRO DE PERMISSÃO ao salvar CSV em '{caminho_completo_csv_destino}'. Verifique se o arquivo está aberto ou se há permissão.")
except Exception as e:
    logging.error(f"!!! ERRO GERAL INESPERADO: {e}")
    import traceback
    traceback.print_exc()

logging.info("--- FIM DO SCRIPT ---")

2025-05-14 15:44:43,884 - INFO - -----------------------------------------------------
2025-05-14 15:44:43,885 - INFO - Processando 'Transportadora (Qtd de caixas, Km).xlsx' para gerar 'Transportadora.csv'.
2025-05-14 15:44:43,887 - INFO - Selecionando colunas, limpando N/A, DIVIDINDO Quantidade, LIMPANDO Distância, formatando datas, etc.
2025-05-14 15:44:43,887 - INFO - -----------------------------------------------------


2025-05-14 15:44:43,894 - INFO - [OK] Pasta de destino CSV verificada/criada: 'G:\Meu Drive\Relatorio Oficial 2025\Drive\Custos Transportadora'
2025-05-14 15:44:43,967 - INFO - Lendo arquivo Excel: Transportadora (Qtd de caixas, Km).xlsx...
2025-05-14 15:44:44,136 - INFO - Leitura inicial completa: 1417 linhas.
2025-05-14 15:44:44,136 - INFO - Selecionando colunas: ['Número da Rota', 'Status', 'Quantidade de Pacotes', 'Data de Atualização', 'Distância', 'Transportadora']...
2025-05-14 15:44:44,136 - INFO - Colunas mantidas inicialmente: ['Número da Rota', 'Status', 'Quantidade de Pacotes', 'Data de Atualização', 'Distância', 'Transportadora']
2025-05-14 15:44:44,136 - INFO - Realizando limpeza básica (espaços, linhas vazias, N/A)...
2025-05-14 15:44:44,150 - INFO - Substituindo 'N/A' por vazio...
2025-05-14 15:44:44,156 - INFO - Linhas após limpeza: 1417
2025-05-14 15:44:44,156 - INFO - Dividindo coluna 'Quantidade de Pacotes'...
2025-05-14 15:44:44,156 - INFO -  -> Colunas 'Qtd Previs

# Separação
Verificar

In [10]:
# prompt: ta errado ainda, faz pela data mesmo, do dia 28 de dez 2024, até today(), a planilha tem q vim com essas datas, mas n pode se repetir, se não, vai fica os mesmo dados.

import os
import pandas as pd
from datetime import datetime, timedelta

# 🗂 Caminho onde deseja salvar o arquivo final (MODIFIQUE CONFORME SUA MÁQUINA)
caminho_pasta_local = r"G:\Meu Drive\Relatorio Oficial 2025\Drive\Pedidos Separados"

# 🛠 Criar a pasta caso não exista
if not os.path.exists(caminho_pasta_local):
    os.makedirs(caminho_pasta_local)
    print(f"Pasta criada: {caminho_pasta_local}")

# 📂 Pasta onde os arquivos XLSM estão no Google Drive
pasta_drive = r"G:.shortcut-targets-by-id\1NpLOQC_eCJ4vFpBiP54a5AhOorhBC0kZ\Logística\Acompanhamento separação"

# 📊 Lista para armazenar os DataFrames de cada arquivo
dfs = []
datas_vistas = set()  # cria um conjunto, que é uma estrutura de dados que armazena valores únicos e não ordenados.
# Isso significa que ele não permite elementos duplicados e pode ser útil para rastrear valores distintos.


# 🔄 Loop para processar os arquivos
data_inicio = datetime(2024, 12, 28)
data_fim = datetime.today()

for i in range(1, 13):  # Ciclos de 1 a 12
    nome_arquivo = f'Acompanhamento de separação Ciclo {i:02}-25.xlsm'
    caminho_arquivo = os.path.join(pasta_drive, nome_arquivo)

    # Verifica se o arquivo existe antes de tentar abrir
    if os.path.exists(caminho_arquivo):
        try:
            df = pd.read_excel(caminho_arquivo, sheet_name="Base de dados", engine='openpyxl')

            # Converte a coluna de data para datetime, ignorando erros
            if 'Data' in df.columns:
                df['Data'] = pd.to_datetime(df['Data'], errors='coerce')

                # 🔍 Remove registros que possuem uma data já vista anteriormente
                df_sem_repetidos = df[~df['Data'].isin(datas_vistas)]

                # Atualiza as datas já vistas
                datas_vistas.update(df_sem_repetidos['Data'].unique())

                # Filtra a data no range definido
                df_filtrado = df_sem_repetidos[(df_sem_repetidos['Data'] >= data_inicio) & (df_sem_repetidos['Data'] <= data_fim)]

                dfs.append(df_filtrado)
                print(f"✔️ Arquivo {nome_arquivo} processado com datas únicas!")

        except Exception as e:
            print(f"❌ Erro ao ler o arquivo {nome_arquivo}: {e}")
    else:
        print(f"⚠️ Arquivo {nome_arquivo} não encontrado.")

# 🏁 Concatenando os DataFrames
if dfs:
    df_final = pd.concat(dfs, ignore_index=True)
    print(f"🔄 DataFrames concatenados! Total de linhas: {df_final.shape[0]}")

    # 📥 Caminho do arquivo final na sua máquina
    caminho_arquivo_local = os.path.join(caminho_pasta_local, "Acompanhamento_separação_completo.xlsx")

    # 💾 Salvando na sua máquina
    df_final.to_excel(caminho_arquivo_local, index=False, engine='openpyxl')

    print(f"✅ Arquivo final salvo em: {caminho_arquivo_local}")
else:
    print("⚠️ Nenhum arquivo foi processado. Verifique os caminhos dos arquivos.")



✔️ Arquivo Acompanhamento de separação Ciclo 01-25.xlsm processado com datas únicas!
✔️ Arquivo Acompanhamento de separação Ciclo 02-25.xlsm processado com datas únicas!
✔️ Arquivo Acompanhamento de separação Ciclo 03-25.xlsm processado com datas únicas!
✔️ Arquivo Acompanhamento de separação Ciclo 04-25.xlsm processado com datas únicas!
✔️ Arquivo Acompanhamento de separação Ciclo 05-25.xlsm processado com datas únicas!
✔️ Arquivo Acompanhamento de separação Ciclo 06-25.xlsm processado com datas únicas!
✔️ Arquivo Acompanhamento de separação Ciclo 07-25.xlsm processado com datas únicas!
✔️ Arquivo Acompanhamento de separação Ciclo 08-25.xlsm processado com datas únicas!
⚠️ Arquivo Acompanhamento de separação Ciclo 09-25.xlsm não encontrado.
⚠️ Arquivo Acompanhamento de separação Ciclo 10-25.xlsm não encontrado.
⚠️ Arquivo Acompanhamento de separação Ciclo 11-25.xlsm não encontrado.
⚠️ Arquivo Acompanhamento de separação Ciclo 12-25.xlsm não encontrado.
🔄 DataFrames concatenados! Total

In [11]:
import os
import pandas as pd
from datetime import datetime, timedelta
import logging
import re # Importar a biblioteca re

# --- Configurações ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 🗂 Caminho onde deseja salvar o arquivo final
caminho_pasta_local = r"G:\Meu Drive\Relatorio Oficial 2025\Drive\Pedidos Separados"
nome_arquivo_final = "Acompanhamento_separação_completo_formatado.xlsx" # Nome do arquivo de saída
caminho_arquivo_local = os.path.join(caminho_pasta_local, nome_arquivo_final)

# 📂 Pasta onde os arquivos XLSM de origem estão
pasta_drive = r"G:.shortcut-targets-by-id\1NpLOQC_eCJ4vFpBiP54a5AhOorhBC0kZ\Logística\Acompanhamento separação"

# 📅 Período de datas para filtrar
data_inicio = datetime(2024, 12, 28)
data_fim = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)
logging.info(f"Período de filtro: {data_inicio.strftime('%Y-%m-%d')} a {data_fim.strftime('%Y-%m-%d')}")

def padronizar_responsavel(nome_responsavel):
    """
    Padroniza o nome do responsável:
    - Corrige nomes específicos ('deigo' -> 'Diego', 'Miqueias' -> 'Miquieias').
    - Remove espaços extras no início/fim.
    - Converte para minúsculas.
    - Capitaliza a primeira letra de cada palavra.
    """
    if pd.isna(nome_responsavel) or not isinstance(nome_responsavel, str) or nome_responsavel.strip() == '':
        return '' # Retorna string vazia para nulos, não-strings ou strings vazias

    nome_processado = str(nome_responsavel).strip()

    if 'deigo' in nome_processado.lower():
        nome_processado = re.sub(r'deigo', 'Diego', nome_processado, flags=re.IGNORECASE)

    if 'miqueias' in nome_processado.lower():
        nome_processado = re.sub(r'miqueias', 'Miquieias', nome_processado, flags=re.IGNORECASE)

    nome_padronizado = ' '.join(word.capitalize() for word in nome_processado.lower().split())

    if 'Diego' in nome_padronizado and 'deigo' in str(nome_responsavel).strip().lower():
        nome_padronizado = re.sub(r'Deigo', 'Diego', nome_padronizado, flags=re.IGNORECASE)

    if 'Miquieias' in nome_padronizado and 'miqueias' in str(nome_responsavel).strip().lower():
        nome_padronizado = re.sub(r'Miqueias', 'Miquieias', nome_padronizado, flags=re.IGNORECASE)

    return nome_padronizado

# --- Processamento ---
try:
    os.makedirs(caminho_pasta_local, exist_ok=True)
    logging.info(f"Pasta de destino verificada/criada: {caminho_pasta_local}")
except Exception as e:
    logging.error(f"Erro ao verificar/criar pasta de destino: {e}")
    exit()

dfs = []
datas_vistas = set()

logging.info("Iniciando processamento dos arquivos de ciclo...")
for i in range(1, 13):
    nome_arquivo = f'Acompanhamento de separação Ciclo {i:02}-25.xlsm'
    caminho_arquivo = os.path.join(pasta_drive, nome_arquivo)
    logging.info(f"Verificando arquivo: {nome_arquivo}...")

    if os.path.exists(caminho_arquivo):
        try:
            logging.info(f"  -> Lendo aba 'Base de dados'...")
            df = pd.read_excel(caminho_arquivo, sheet_name="Base de dados", engine='openpyxl')
            logging.info(f"  -> Leitura OK: {len(df)} linhas.")

            if 'Data' in df.columns:
                df['Data'] = pd.to_datetime(df['Data'].astype(str), errors='coerce')
                linhas_antes_nan = len(df)
                df.dropna(subset=['Data'], inplace=True)
                linhas_depois_nan = len(df)
                if linhas_antes_nan > linhas_depois_nan:
                    logging.warning(f"  -> {linhas_antes_nan - linhas_depois_nan} linhas removidas por terem data inválida ou vazia.")

                if not df.empty:
                    df_copia = df.copy()
                    df_sem_repetidos = df_copia[~df_copia['Data'].isin(datas_vistas)]

                    if not df_sem_repetidos.empty:
                        datas_vistas.update(df_sem_repetidos['Data'].unique())
                        logging.debug(f"  -> Filtrando datas entre {data_inicio.date()} e {data_fim.date()}...")
                        df_filtrado = df_sem_repetidos[
                            (df_sem_repetidos['Data'] >= data_inicio) &
                            (df_sem_repetidos['Data'] <= data_fim)
                        ].copy() # .copy() aqui é uma boa prática

                        if not df_filtrado.empty:
                            # -------> LOCAL CORRETO PARA PADRONIZAR A COLUNA 'Responsável' <-------
                            nome_coluna_responsavel = 'Responsável' # Defina o nome exato da sua coluna aqui
                            if nome_coluna_responsavel in df_filtrado.columns:
                                logging.info(f"  -> Padronizando coluna '{nome_coluna_responsavel}' no arquivo {nome_arquivo}...")
                                # Criar uma cópia para evitar SettingWithCopyWarning ao aplicar a função
                                df_filtrado_para_modificar = df_filtrado.copy()
                                df_filtrado_para_modificar[nome_coluna_responsavel] = df_filtrado_para_modificar[nome_coluna_responsavel].apply(padronizar_responsavel)
                                df_filtrado = df_filtrado_para_modificar # Atribui o DataFrame modificado de volta
                                logging.info(f"  -> Coluna '{nome_coluna_responsavel}' padronizada.")
                            else:
                                logging.warning(f"  -> Coluna '{nome_coluna_responsavel}' não encontrada no arquivo {nome_arquivo}. Não será padronizada.")
                            # ---------------------------------------------------------------------

                            dfs.append(df_filtrado)
                            logging.info(f"  -> ✔️ {len(df_filtrado)} linhas adicionadas de {nome_arquivo} (datas únicas no período).")
                        else:
                            logging.info(f"  -> Nenhuma linha de {nome_arquivo} está dentro do período após deduplicação.")
                    else:
                        logging.info(f"  -> Todas as datas válidas de {nome_arquivo} já foram vistas em arquivos anteriores.")
            else:
                logging.warning(f"  -> Coluna 'Data' não encontrada no arquivo {nome_arquivo}. Arquivo ignorado.")
        except Exception as e:
            logging.error(f"  -> ❌ Erro ao ler ou processar o arquivo {nome_arquivo}: {e}")
            # import traceback # Descomente para depuração detalhada
            # traceback.print_exc() # Descomente para depuração detalhada
    else:
        logging.warning(f"  -> ⚠️ Arquivo não encontrado: {caminho_arquivo}")

logging.info("Processamento dos arquivos de ciclo concluído.")
if dfs:
    logging.info("Concatenando resultados...")
    df_final = pd.concat(dfs, ignore_index=True)
    logging.info(f"🔄 DataFrames concatenados! Total de linhas final: {df_final.shape[0]}")

    if 'Data' in df_final.columns:
        try:
            logging.info("Formatando coluna 'Data' para AAAA-MM-DD no DataFrame final...")
            df_final['Data'] = pd.to_datetime(df_final['Data'], errors='coerce')
            df_final['Data'] = df_final['Data'].dt.strftime('%Y-%m-%d')
            df_final['Data'].fillna('', inplace=True)
            logging.info("Coluna 'Data' formatada com sucesso.")
        except Exception as format_e:
            logging.error(f"Erro ao formatar a coluna 'Data' final: {format_e}")
            logging.warning("A coluna 'Data' pode conter a hora no arquivo salvo.")

    try:
        logging.info(f"Salvando arquivo final em: {caminho_arquivo_local}...")
        df_final.to_excel(caminho_arquivo_local, index=False, engine='openpyxl')
        logging.info(f"✅ Arquivo final salvo com sucesso!")
    except PermissionError:
        logging.error(f"!!! ERRO DE PERMISSÃO ao salvar Excel em '{caminho_arquivo_local}'. Verifique se o arquivo está aberto ou se há permissão de escrita.")
    except Exception as e:
        logging.error(f"!!! ERRO ao salvar o arquivo Excel final: {e}")
else:
    logging.warning("⚠️ Nenhum dado foi processado ou filtrado dos arquivos de ciclo. Nenhum arquivo final foi gerado.")

logging.info("--- FIM DO SCRIPT ---")

2025-06-06 10:33:50,770 - INFO - Período de filtro: 2024-12-28 a 2025-06-06
2025-06-06 10:33:50,795 - INFO - Pasta de destino verificada/criada: G:\Meu Drive\Relatorio Oficial 2025\Drive\Pedidos Separados


2025-06-06 10:33:50,799 - INFO - Iniciando processamento dos arquivos de ciclo...
2025-06-06 10:33:50,803 - INFO - Verificando arquivo: Acompanhamento de separação Ciclo 01-25.xlsm...
2025-06-06 10:33:50,811 - INFO -   -> Lendo aba 'Base de dados'...
2025-06-06 10:33:52,045 - INFO -   -> Leitura OK: 470 linhas.
2025-06-06 10:33:52,059 - WARNING -   -> 206 linhas removidas por terem data inválida ou vazia.
2025-06-06 10:33:52,059 - INFO -   -> Padronizando coluna 'Responsável' no arquivo Acompanhamento de separação Ciclo 01-25.xlsm...
2025-06-06 10:33:52,071 - INFO -   -> Coluna 'Responsável' padronizada.
2025-06-06 10:33:52,079 - INFO -   -> ✔️ 264 linhas adicionadas de Acompanhamento de separação Ciclo 01-25.xlsm (datas únicas no período).
2025-06-06 10:33:52,079 - INFO - Verificando arquivo: Acompanhamento de separação Ciclo 02-25.xlsm...
2025-06-06 10:33:52,088 - INFO -   -> Lendo aba 'Base de dados'...
2025-06-06 10:33:54,374 - INFO -   -> Leitura OK: 362 linhas.
2025-06-06 10:33:5

### Enviar para o bigquery

In [12]:
import os
import pandas as pd
from datetime import datetime, timedelta
import logging
import re
import unicodedata # Importado para remover acentos

# --- Configurações ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

caminho_pasta_local = r"G:\Meu Drive\Relatorio Oficial 2025\Drive\Pedidos Separados"
nome_arquivo_final = "Acompanhamento_separação_completo_formatado.csv"
caminho_arquivo_local = os.path.join(caminho_pasta_local, nome_arquivo_final)

pasta_drive = r"G:.shortcut-targets-by-id\1NpLOQC_eCJ4vFpBiP54a5AhOorhBC0kZ\Logística\Acompanhamento separação"

data_inicio = datetime(2024, 12, 28)
data_fim = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)
logging.info(f"Período de filtro: {data_inicio.strftime('%Y-%m-%d')} a {data_fim.strftime('%Y-%m-%d')}")

def padronizar_responsavel(nome_responsavel):
    if pd.isna(nome_responsavel) or not isinstance(nome_responsavel, str) or nome_responsavel.strip() == '':
        return ''
    nome_processado = str(nome_responsavel).strip()
    if 'deigo' in nome_processado.lower():
        nome_processado = re.sub(r'deigo', 'Diego', nome_processado, flags=re.IGNORECASE)
    if 'miqueias' in nome_processado.lower():
        nome_processado = re.sub(r'miqueias', 'Miquieias', nome_processado, flags=re.IGNORECASE)
    nome_padronizado = ' '.join(word.capitalize() for word in nome_processado.lower().split())
    if 'Diego' in nome_padronizado and 'deigo' in str(nome_responsavel).strip().lower():
        nome_padronizado = re.sub(r'Deigo', 'Diego', nome_padronizado, flags=re.IGNORECASE)
    if 'Miquieias' in nome_padronizado and 'miqueias' in str(nome_responsavel).strip().lower():
        nome_padronizado = re.sub(r'Miqueias', 'Miquieias', nome_padronizado, flags=re.IGNORECASE)
    return nome_padronizado

def corrigir_nome_coluna(nome_coluna):
    mapa_nomes_completos = {
        'ResponsÃ¡vel': 'Responsável', 'RegiÃ£o': 'Região',
        'NÂ° de Pedidos': 'N° de Pedidos', 'NÂ° de Itens': 'N° de Itens',
        'InÃ­cio Picking': 'Início Picking', 'Fim Picking': 'Fim Picking',
        'PrevisÃ£o de tÃ©rmino': 'Previsão de término',
        'Tempo mÃ©dio por item': 'Tempo médio por item',
        'Tempo mÃ©dio por Pedido': 'Tempo médio por Pedido',
        'HorÃ¡rio de almoÃ§o': 'Horário de almoço'
    }
    if nome_coluna in mapa_nomes_completos:
        return mapa_nomes_completos[nome_coluna]
    nome_corrigido = nome_coluna
    substituicoes_caracteres = {
        'NÂ°': 'N°', 'Ã¡': 'á', 'Ã©': 'é', 'Ã­': 'í', 'Ã³': 'ó', 'Ãº': 'ú',
        'Ã‚': 'Â', 'ÃŠ': 'Ê', 'Ã”': 'Ô', 'Ã¢': 'â', 'Ãª': 'ê', 'Ã´': 'ô',
        'Ã£': 'ã', 'Ãµ': 'õ', 'Ã§': 'ç', 'Â°': '°', 'Âº': 'º', 'Âª': 'ª',
        'Ãü': 'ü', 'Ã': 'Á', 'Ã‰': 'É', 'Ã': 'Í', 'Ã“': 'Ó', 'Ãš': 'Ú',
        'Ãƒ': 'Ã', 'Ã•': 'Õ', 'Ã‡': 'Ç', 'Ãœ': 'Ü'
    }
    for problematica, correta in substituicoes_caracteres.items():
        nome_corrigido = nome_corrigido.replace(problematica, correta)
    return nome_corrigido

# NOVA FUNÇÃO para remover acentos
def remover_acentos(texto):
    if not isinstance(texto, str):
        return texto
    nfkd_form = unicodedata.normalize('NFKD', texto)
    return "".join([c for c in nfkd_form if not unicodedata.combining(c)])

# NOVA FUNÇÃO para higienizar nomes para o BigQuery
def higienizar_nome_para_bigquery(nome_coluna):
    if not isinstance(nome_coluna, str) or not nome_coluna.strip():
        # Para nomes de coluna inválidos ou vazios, cria um placeholder
        return f"coluna_invalida_{abs(hash(str(nome_coluna))) % 10000}"

    # 1. Remove acentos (ex: "Região" -> "Regiao")
    nome_sem_acentos = remover_acentos(nome_coluna)
    
    processado = nome_sem_acentos
    # 2. Substituições específicas (ex: "N° de" -> "Num_de", "N°" -> "No")
    processado = processado.replace('N° de', 'Num_de') 
    processado = processado.replace('N°', 'No')      
    processado = processado.replace('°', 'o') # Para qualquer '°' restante

    # 3. Substitui qualquer sequência de caracteres que NÃO seja letra ou número por um único underscore
    processado = re.sub(r'[^a-zA-Z0-9_]+', '_', processado) # Underscore já é permitido, então incluímos na negação

    # 4. Remove underscores no início ou no fim
    processado = processado.strip('_')
    
    # 5. Se o nome ficou vazio após as transformações (ex: nome original era "---")
    if not processado:
        nome_base_original = re.sub(r'[^a-zA-Z0-9]', '', nome_coluna) # Usa o nome original da coluna
        if nome_base_original:
            processado = 'col_' + nome_base_original[:20].lower() # Cria um nome a partir do original
        else: # Se o original também não tinha letras/números
            processado = f'col_gen_{abs(hash(nome_coluna)) % 10000}'

    # 6. Garante que não comece com um número (BigQuery exige começar com letra ou underscore)
    if processado and processado[0].isdigit(): # Verifica se processado não é vazio antes de indexar
        processado = '_' + processado
        
    return processado.lower() # Padroniza para minúsculas para consistência

# --- Processamento ---
try:
    os.makedirs(caminho_pasta_local, exist_ok=True)
    logging.info(f"Pasta de destino verificada/criada: {caminho_pasta_local}")
except Exception as e:
    logging.error(f"Erro ao verificar/criar pasta de destino: {e}")
    exit()

dfs = []
datas_vistas = set()

# Nome original da coluna 'Responsável' (após correção de codificação, antes da higienização BQ)
nome_coluna_responsavel_corrigido = 'Responsável'
# Obter o nome higienizado para usar nas buscas e manipulações
nome_coluna_responsavel_bq = higienizar_nome_para_bigquery(nome_coluna_responsavel_corrigido)

# Nome original da coluna de Data (geralmente não muda com higienização se for só 'Data')
nome_coluna_data_corrigido = 'Data'
nome_coluna_data_bq = higienizar_nome_para_bigquery(nome_coluna_data_corrigido)


logging.info("Iniciando processamento dos arquivos de ciclo...")
for i in range(1, 13):
    nome_arquivo = f'Acompanhamento de separação Ciclo {i:02}-25.xlsm'
    caminho_arquivo = os.path.join(pasta_drive, nome_arquivo)
    logging.info(f"Verificando arquivo: {nome_arquivo}...")

    if os.path.exists(caminho_arquivo):
        try:
            logging.info(f"  -> Lendo aba 'Base de dados'...")
            df = pd.read_excel(caminho_arquivo, sheet_name="Base de dados", engine='openpyxl')
            logging.info(f"  -> Leitura OK: {len(df)} linhas.")

            # Etapa 1: Corrigir problemas de codificação nos nomes das colunas
            df.columns = [corrigir_nome_coluna(col) for col in df.columns]
            logging.info(f"  -> Nomes das colunas com codificação corrigida para o arquivo {nome_arquivo}.")
            
            # Etapa 2: Higienizar nomes das colunas para compatibilidade com BigQuery
            df.columns = [higienizar_nome_para_bigquery(col) for col in df.columns]
            logging.info(f"  -> Nomes das colunas higienizados para BigQuery no arquivo {nome_arquivo}.")

            if nome_coluna_data_bq in df.columns:
                df[nome_coluna_data_bq] = pd.to_datetime(df[nome_coluna_data_bq].astype(str), errors='coerce', dayfirst=True)
                linhas_antes_nan = len(df)
                df.dropna(subset=[nome_coluna_data_bq], inplace=True)
                linhas_depois_nan = len(df)
                if linhas_antes_nan > linhas_depois_nan:
                    logging.warning(f"  -> {linhas_antes_nan - linhas_depois_nan} linhas removidas por terem data inválida ou vazia na coluna '{nome_coluna_data_bq}'.")

                if not df.empty:
                    df_copia = df.copy()
                    # Usar o nome da coluna de data higienizado para a deduplicação
                    df_sem_repetidos = df_copia[~df_copia[nome_coluna_data_bq].isin(datas_vistas)]

                    if not df_sem_repetidos.empty:
                        datas_vistas.update(df_sem_repetidos[nome_coluna_data_bq].unique())
                        df_filtrado = df_sem_repetidos[
                            (df_sem_repetidos[nome_coluna_data_bq] >= data_inicio) &
                            (df_sem_repetidos[nome_coluna_data_bq] <= data_fim)
                        ].copy()

                        if not df_filtrado.empty:
                            if nome_coluna_responsavel_bq in df_filtrado.columns:
                                logging.info(f"  -> Padronizando coluna '{nome_coluna_responsavel_bq}' no arquivo {nome_arquivo}...")
                                df_filtrado_para_modificar = df_filtrado.copy() # Evitar SettingWithCopyWarning
                                df_filtrado_para_modificar[nome_coluna_responsavel_bq] = df_filtrado_para_modificar[nome_coluna_responsavel_bq].apply(padronizar_responsavel)
                                df_filtrado = df_filtrado_para_modificar
                                logging.info(f"  -> Coluna '{nome_coluna_responsavel_bq}' padronizada.")
                            else:
                                logging.warning(f"  -> Coluna '{nome_coluna_responsavel_bq}' não encontrada no arquivo {nome_arquivo} após higienização. Não será padronizada.")
                            
                            dfs.append(df_filtrado)
                            logging.info(f"  -> ✔️ {len(df_filtrado)} linhas adicionadas de {nome_arquivo} (datas únicas no período).")
                        else:
                            logging.info(f"  -> Nenhuma linha de {nome_arquivo} está dentro do período após deduplicação.")
                    else:
                        logging.info(f"  -> Todas as datas válidas de {nome_arquivo} (coluna '{nome_coluna_data_bq}') já foram vistas em arquivos anteriores.")
            else:
                logging.warning(f"  -> Coluna de data '{nome_coluna_data_bq}' não encontrada no arquivo {nome_arquivo} (após higienização). Arquivo ignorado.")
        except Exception as e:
            logging.error(f"  -> ❌ Erro ao ler ou processar o arquivo {nome_arquivo}: {e}")
            import traceback
            traceback.print_exc() # Para depuração mais detalhada do erro
    else:
        logging.warning(f"  -> ⚠️ Arquivo não encontrado: {caminho_arquivo}")

logging.info("Processamento dos arquivos de ciclo concluído.")
if dfs:
    logging.info("Concatenando resultados...")
    df_final = pd.concat(dfs, ignore_index=True)
    logging.info(f"🔄 DataFrames concatenados! Total de linhas final: {df_final.shape[0]}")

    # Usar o nome da coluna de data higienizado para a formatação final
    if nome_coluna_data_bq in df_final.columns:
        try:
            logging.info(f"Formatando coluna '{nome_coluna_data_bq}' para YYYY-MM-DD no DataFrame final...")
            df_final[nome_coluna_data_bq] = pd.to_datetime(df_final[nome_coluna_data_bq], errors='coerce')
            df_final[nome_coluna_data_bq] = df_final[nome_coluna_data_bq].dt.strftime('%Y-%m-%d')
            df_final[nome_coluna_data_bq].fillna('', inplace=True)
            logging.info(f"Coluna '{nome_coluna_data_bq}' formatada com sucesso para YYYY-MM-DD.")
        except Exception as format_e:
            logging.error(f"Erro ao formatar a coluna '{nome_coluna_data_bq}' final: {format_e}")
            logging.warning(f"A coluna '{nome_coluna_data_bq}' pode não estar no formato YYYY-MM-DD no arquivo CSV salvo.")
    else:
        logging.warning(f"Coluna de data '{nome_coluna_data_bq}' não encontrada no DataFrame final para formatação.")

    try:
        logging.info(f"Salvando arquivo final em formato CSV: {caminho_arquivo_local}...")
        df_final.to_csv(caminho_arquivo_local, index=False, sep=',', encoding='utf-8')
        logging.info(f"✅ Arquivo CSV final salvo com sucesso! Nomes de colunas estão higienizados para BigQuery.")
    except PermissionError:
        logging.error(f"!!! ERRO DE PERMISSÃO ao salvar CSV em '{caminho_arquivo_local}'. Verifique se o arquivo está aberto ou se há permissão de escrita.")
    except Exception as e:
        logging.error(f"!!! ERRO ao salvar o arquivo CSV final: {e}")
else:
    logging.warning("⚠️ Nenhum dado foi processado ou filtrado dos arquivos de ciclo. Nenhum arquivo final foi gerado.")

logging.info("--- FIM DO SCRIPT ---")

2025-06-06 10:34:53,569 - INFO - Período de filtro: 2024-12-28 a 2025-06-06
2025-06-06 10:34:53,582 - INFO - Pasta de destino verificada/criada: G:\Meu Drive\Relatorio Oficial 2025\Drive\Pedidos Separados
2025-06-06 10:34:53,587 - INFO - Iniciando processamento dos arquivos de ciclo...
2025-06-06 10:34:53,590 - INFO - Verificando arquivo: Acompanhamento de separação Ciclo 01-25.xlsm...
2025-06-06 10:34:53,597 - INFO -   -> Lendo aba 'Base de dados'...


2025-06-06 10:34:54,687 - INFO -   -> Leitura OK: 470 linhas.
2025-06-06 10:34:54,692 - INFO -   -> Nomes das colunas com codificação corrigida para o arquivo Acompanhamento de separação Ciclo 01-25.xlsm.
2025-06-06 10:34:54,692 - INFO -   -> Nomes das colunas higienizados para BigQuery no arquivo Acompanhamento de separação Ciclo 01-25.xlsm.
C:\Users\Florestas\AppData\Local\Temp\ipykernel_15748\2840626952.py:144: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df[nome_coluna_data_bq] = pd.to_datetime(df[nome_coluna_data_bq].astype(str), errors='coerce', dayfirst=True)
2025-06-06 10:34:54,710 - WARNING -   -> 206 linhas removidas por terem data inválida ou vazia na coluna 'data'.
2025-06-06 10:34:54,717 - INFO -   -> Padronizando coluna 'responsavel' no arquivo Acompanhamento de separação Ciclo 01-25.xlsm...
2025-06-06 10:34:54,724 - INFO -   -> Coluna 'responsavel' padronizada.
2025-06

In [13]:
import os
import pandas as pd
import unicodedata # Para a função limpar_nome_coluna_bq
import re          # Para a função limpar_nome_coluna_bq
import logging
from google.oauth2 import service_account # Para carregar as credenciais do BigQuery
import pandas_gbq # Para usar pandas_gbq.to_gbq()

# --- Configuração do Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Caminhos e Nomes ---
caminho_pasta_dados_separacao = r"G:\Meu Drive\Relatorio Oficial 2025\Drive\Pedidos Separados"
nome_arquivo_csv_separacao = "Acompanhamento_separação_completo_formatado.csv"
caminho_completo_csv_separacao = os.path.join(caminho_pasta_dados_separacao, nome_arquivo_csv_separacao)

# --- CONFIGURAÇÕES DO BIGQUERY ---
PROJECT_ID = 'grupo-florestas-429613'
DATASET_ID = 'VDHUB'
TABLE_NAME_SEPARACAO = 'separacao'
DESTINATION_TABLE_SEPARACAO = f'{DATASET_ID}.{TABLE_NAME_SEPARACAO}'
ARQUIVO_CREDENCIAL_JSON = r'C:\Users\Florestas\Desktop\grupo-florestas-429613-080e980a456d.json'

# --- Nomes das Colunas para Tratamento Específico (APÓS LIMPEZA DE NOMES) ---
# Seu script de preparação higieniza 'Data' para 'data'.
NOME_COLUNA_DATA_PRINCIPAL = 'data'
COLUNAS_NUMERICAS = ['num_de_pedidos', 'num_de_itens'] # Estas serão convertidas para número

# --- Funções Auxiliares ---
def limpar_nome_coluna_bq_final_check(col_name):
    if not isinstance(col_name, str): col_name = str(col_name)
    nfkd_form = unicodedata.normalize('NFKD', col_name)
    ascii_name = "".join([c for c in nfkd_form if not unicodedata.combining(c)])
    lower_name = ascii_name.lower()
    cleaned_name = re.sub(r'\s+', '_', lower_name)
    cleaned_name = re.sub(r'[^\w_]+', '', cleaned_name)
    if cleaned_name and cleaned_name[0].isdigit():
        cleaned_name = '_' + cleaned_name
    if not cleaned_name:
        cleaned_name = f'col_gerada_{abs(hash(col_name))%10000}'
    return cleaned_name

# --- Execução Principal ---
logging.info("-----------------------------------------------------")
logging.info(f"Iniciando envio do CSV '{nome_arquivo_csv_separacao}' para BigQuery '{DESTINATION_TABLE_SEPARACAO}'.")
logging.info(f"Coluna de data '{NOME_COLUNA_DATA_PRINCIPAL}' será enviada como STRING (como está no CSV).")
logging.info(f"Colunas {COLUNAS_NUMERICAS} serão convertidas para numérico.")
logging.info("Outras colunas serão enviadas como STRING.")
logging.info("A tabela no BigQuery será SUBSTITUÍDA.")
logging.info("-----------------------------------------------------")

# 1. Autenticar
if not os.path.exists(ARQUIVO_CREDENCIAL_JSON):
    logging.error(f"!!! ERRO FATAL: Arquivo de Credenciais JSON não encontrado: '{ARQUIVO_CREDENCIAL_JSON}'.")
    exit()
try:
    credentials = service_account.Credentials.from_service_account_file(
        ARQUIVO_CREDENCIAL_JSON,
        scopes=["https://www.googleapis.com/auth/bigquery"]
    )
    logging.info(f"[OK] Credenciais carregadas: {ARQUIVO_CREDENCIAL_JSON}")
except Exception as auth_err:
    logging.error(f"!!! ERRO FATAL: Falha ao carregar credenciais: {auth_err}")
    exit()

# 2. Verifica se o arquivo CSV de origem existe
if not os.path.exists(caminho_completo_csv_separacao):
    logging.error(f"!!! ERRO FATAL: Arquivo CSV de origem não encontrado: '{caminho_completo_csv_separacao}'.")
    exit()

# 3. Lê o CSV e Prepara o DataFrame
df_bq_separacao = None
try:
    logging.info(f"Lendo arquivo CSV: {nome_arquivo_csv_separacao}...")
    df_bq_separacao = pd.read_csv(caminho_completo_csv_separacao, dtype=str, keep_default_na=False, na_values=[''])

    if df_bq_separacao.empty:
        raise ValueError("O arquivo CSV está vazio.")
    logging.info(f"Leitura do CSV OK: {len(df_bq_separacao)} linhas.")

    logging.info("Limpando nomes das colunas para o BigQuery...")
    df_bq_separacao.columns = [limpar_nome_coluna_bq_final_check(col) for col in df_bq_separacao.columns]
    logging.info(f"Nomes das colunas limpos (ex: {list(df_bq_separacao.columns[:5])}...).")

    # NÃO convertemos a coluna de data principal para datetime, ela permanecerá como string.
    if NOME_COLUNA_DATA_PRINCIPAL in df_bq_separacao.columns:
        logging.info(f"Coluna de data principal '{NOME_COLUNA_DATA_PRINCIPAL}' será mantida como STRING (como lida do CSV).")
    else:
        logging.warning(f"Coluna de data principal '{NOME_COLUNA_DATA_PRINCIPAL}' não encontrada no DataFrame.")

    # Converte as colunas numéricas especificadas
    logging.info(f"Convertendo colunas {COLUNAS_NUMERICAS} para tipo numérico...")
    for col_num in COLUNAS_NUMERICAS:
        if col_num in df_bq_separacao.columns:
            if df_bq_separacao[col_num].dtype == 'object': # Se for string
                 df_bq_separacao[col_num] = df_bq_separacao[col_num].str.replace(',', '.', regex=False)
            df_bq_separacao[col_num] = pd.to_numeric(df_bq_separacao[col_num], errors='coerce')
            num_nulos_num = df_bq_separacao[col_num].isna().sum()
            if num_nulos_num > 0:
                logging.warning(f"Coluna '{col_num}': {num_nulos_num} valores não convertidos para número e serão NULOS.")
            logging.info(f"Coluna '{col_num}' convertida para numérico.")
        else:
            logging.warning(f"Coluna numérica '{col_num}' não encontrada no DataFrame.")

    # Define o esquema da tabela para o BigQuery
    table_schema = []
    for col_name in df_bq_separacao.columns:
        if col_name in COLUNAS_NUMERICAS:
            # Vamos assumir INTEGER. Se precisar de decimais, use 'FLOAT' ou 'NUMERIC'.
            table_schema.append({'name': col_name, 'type': 'INTEGER'})
        # A coluna de data principal e todas as outras não listadas como numéricas serão STRING
        else:
            table_schema.append({'name': col_name, 'type': 'STRING'})
    logging.info(f"Schema definido para o BigQuery (coluna de data principal como STRING): {table_schema}")

    logging.info("Preparação do DataFrame e schema para envio ao BigQuery concluída.")

except Exception as e:
    logging.error(f"!!! ERRO FATAL durante leitura ou preparação do CSV: {e}")
    import traceback
    traceback.print_exc()
    exit()

# 4. Envia o DataFrame para o BigQuery
if df_bq_separacao is not None and not df_bq_separacao.empty:
    logging.info(f"Enviando {len(df_bq_separacao)} linhas para BigQuery: '{PROJECT_ID}:{DESTINATION_TABLE_SEPARACAO}' (Substituindo)...")
    try:
        pandas_gbq.to_gbq(
            df_bq_separacao,
            destination_table=DESTINATION_TABLE_SEPARACAO,
            project_id=PROJECT_ID,
            credentials=credentials,
            if_exists='replace',
            table_schema=table_schema, # Fornecendo o schema para garantir os tipos
            progress_bar=True
        )
        logging.info(f"✅ SUCESSO! Dados enviados e tabela '{DESTINATION_TABLE_SEPARACAO}' foi substituída no BigQuery.")
        logging.info(f"Verifique o esquema da tabela no BigQuery: '{NOME_COLUNA_DATA_PRINCIPAL}' deve ser STRING, {COLUNAS_NUMERICAS} devem ser INTEGER.")
    except Exception as e:
        logging.error(f"!!! ERRO AO ENVIAR PARA O BIGQUERY: {e}")
        logging.error("Verifique: Permissões no IAM, PROJECT_ID, DESTINATION_TABLE, API Ativa.")
        import traceback
        traceback.print_exc()
else:
    logging.error("DataFrame está vazio ou não foi definido. Nenhum dado enviado.")

logging.info(f"--- FIM DO SCRIPT DE ENVIO PARA BIGQUERY ({DESTINATION_TABLE_SEPARACAO}) ---")

2025-06-06 10:35:53,809 - INFO - -----------------------------------------------------
2025-06-06 10:35:53,811 - INFO - Iniciando envio do CSV 'Acompanhamento_separação_completo_formatado.csv' para BigQuery 'VDHUB.separacao'.
2025-06-06 10:35:53,816 - INFO - Coluna de data 'data' será enviada como STRING (como está no CSV).
2025-06-06 10:35:53,817 - INFO - Colunas ['num_de_pedidos', 'num_de_itens'] serão convertidas para numérico.
2025-06-06 10:35:53,821 - INFO - Outras colunas serão enviadas como STRING.
2025-06-06 10:35:53,821 - INFO - A tabela no BigQuery será SUBSTITUÍDA.
2025-06-06 10:35:53,825 - INFO - -----------------------------------------------------
2025-06-06 10:35:53,967 - INFO - [OK] Credenciais carregadas: C:\Users\Florestas\Desktop\grupo-florestas-429613-080e980a456d.json
2025-06-06 10:35:53,975 - INFO - Lendo arquivo CSV: Acompanhamento_separação_completo_formatado.csv...
2025-06-06 10:35:54,035 - INFO - Leitura do CSV OK: 1571 linhas.
2025-06-06 10:35:54,044 - INFO -

# Devoluções e Cancelamentos
Salvo em csv para subir para bigquery

In [14]:
# Configuração básica de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Caminhos ---
excel_folder_path = r"C:\Users\Florestas\Desktop\Relatorio Oficial 2025\Excel - Pedidos"
excel_file_name = "Monitoramento de Risco (Inicio Abril_24).xlsx"
excel_file_path = os.path.join(excel_folder_path, excel_file_name)

csv_output_folder = r"C:\Users\Florestas\Desktop\Relatorio Oficial 2025\Devoluções e Cancelamentos"
csv_file_name = "Monitoramento de Risco (Inicio Abril_24)_limpo.csv" # Adicionei '_limpo' ao nome
csv_file_path = os.path.join(csv_output_folder, csv_file_name)

# --- Função para Limpar Nomes de Colunas ---
def limpar_nome_coluna(col_name):
    if not isinstance(col_name, str): col_name = str(col_name)
    nfkd_form = unicodedata.normalize('NFKD', col_name)
    ascii_name = nfkd_form.encode('ASCII', 'ignore').decode('ASCII')
    lower_name = ascii_name.lower()
    cleaned_name = re.sub(r'\s+', '_', lower_name)
    cleaned_name = re.sub(r'[^\w_]+', '', cleaned_name)
    cleaned_name = re.sub(r'_+', '_', cleaned_name)
    cleaned_name = cleaned_name.strip('_')
    if cleaned_name and cleaned_name[0].isdigit(): cleaned_name = '_' + cleaned_name
    if not cleaned_name: cleaned_name = 'coluna_vazia'
    return cleaned_name

# --- Processo ---
logging.info(f"Iniciando conversão de '{excel_file_name}' para CSV...")
logging.info("Limpando Nomes de Coluna e Quebras de Linha internas.")

# 1. Garante que a pasta de destino exista
try:
    os.makedirs(csv_output_folder, exist_ok=True)
    logging.info(f"[OK] Pasta de destino verificada/criada: {csv_output_folder}")
except Exception as e:
     logging.error(f"⚠️ Erro ao criar pasta de destino '{csv_output_folder}': {e}")

# 2. Verifica se o arquivo de origem existe
if not os.path.exists(excel_file_path):
    logging.error(f"❌ ERRO FATAL: Arquivo Excel não encontrado em: {excel_file_path}")
    exit()

try:
    # 3. Ler o arquivo Excel
    logging.info(f"Lendo arquivo Excel: '{excel_file_name}'...")
    df = pd.read_excel(excel_file_path, dtype=str)
    logging.info(f"Leitura concluída: {len(df)} linhas.")

    if df.empty:
        logging.warning("Planilha Excel está vazia. Arquivo CSV será gerado vazio.")
    else:
        # 4. Limpeza Geral e Específica
        logging.info("Realizando limpeza (espaços, NAs, quebras de linha internas)...")
        df.dropna(how='all', inplace=True) # Remove linhas totalmente vazias

        for col in df.columns:
            # Aplica apenas a colunas que são de texto/objeto
            if pd.api.types.is_object_dtype(df[col]):
                logging.debug(f"Limpando coluna: {col}")
                # Remove espaços inicio/fim
                df[col] = df[col].str.strip()
                # !!! IMPORTANTE: Substitui quebras de linha (\n ou \r\n) por espaço !!!
                df[col] = df[col].astype(str).str.replace('\r\n', ' ', regex=False).str.replace('\n', ' ', regex=False)

        # Substitui 'N/A' (string) por vazio '' (se necessário)
        # df.replace('N/A', '', inplace=True)
        # Preenche nulos restantes (NaN) com vazio ''
        df.fillna('', inplace=True)
        logging.info("Limpeza de dados concluída.")

        # 5. Limpar Nomes das Colunas (ANTES DE SALVAR)
        logging.info("Limpando nomes das colunas...")
        df.columns = [limpar_nome_coluna(col) for col in df.columns]
        logging.info(f"Nomes das colunas limpos (ex: {list(df.columns[:5])}).")

    # 6. Salvar o DataFrame como CSV
    logging.info(f"Salvando como CSV em: '{csv_file_path}'...")
    df.to_csv(
        csv_file_path,
        index=False,
        sep=',',            # Separador vírgula
        encoding='utf-8',   # UTF-8 padrão
        quoting=csv.QUOTE_ALL, # <<< MANTIDO: Coloca aspas em TODOS os campos
        quotechar='"',      # Aspas duplas
        doublequote=True    # Escapa aspas internas como ""
    )

    print(f"✅ Arquivo CSV salvo com sucesso em: {csv_file_path}") # Sua mensagem original

except FileNotFoundError:
    print(f"Erro: Arquivo Excel não encontrado em: {excel_file_path}") # Sua mensagem original
except ImportError:
    print("Erro: Biblioteca necessária não instalada. Tente 'pip install pandas openpyxl'")
except Exception as e:
    print(f"Ocorreu um erro inesperado ao processar o arquivo: {e}") # Sua mensagem original
    import traceback
    traceback.print_exc()

logging.info("--- FIM DO SCRIPT ---")

2025-06-05 08:35:17,115 - INFO - Iniciando conversão de 'Monitoramento de Risco (Inicio Abril_24).xlsx' para CSV...
2025-06-05 08:35:17,117 - INFO - Limpando Nomes de Coluna e Quebras de Linha internas.
2025-06-05 08:35:17,117 - INFO - [OK] Pasta de destino verificada/criada: C:\Users\Florestas\Desktop\Relatorio Oficial 2025\Devoluções e Cancelamentos
2025-06-05 08:35:17,117 - INFO - Lendo arquivo Excel: 'Monitoramento de Risco (Inicio Abril_24).xlsx'...
c:\Users\Florestas\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Cell H163 is marked as a date but the serial value 6703797 is outside the limits for dates. The cell will be treated as an error.
  for idx, row in parser.parse():
c:\Users\Florestas\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Cell H1431 is marked as a date but the serial value 6183717 is outside the limits for dates. The cell will be treated as an error.
  for idx, row in parser.parse():
2025-06-05 08:35:1

✅ Arquivo CSV salvo com sucesso em: C:\Users\Florestas\Desktop\Relatorio Oficial 2025\Devoluções e Cancelamentos\Monitoramento de Risco (Inicio Abril_24)_limpo.csv


# Cortar Captação

In [8]:
# Script para CRIAR dados (baseado na imagem), formatar data AAAA-MM-DD
# e salvar como CSV na pasta especificada.

import pandas as pd
import os
import logging
import csv

# --- Configurações ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Caminho de Destino LOCAL para o CSV ---
caminho_pasta_destino_csv = r'C:\Users\Florestas\Desktop\Relatorio Oficial 2025\Excel - Pedidos\Pedidos'
# Nome do arquivo CSV de saída
nome_arquivo_csv_destino = 'CorteCaptacao.csv' # Nome sugestivo
caminho_completo_csv_destino = os.path.join(caminho_pasta_destino_csv, nome_arquivo_csv_destino)

# --- Dados (Baseados na imagem que você forneceu) ---
dados = {
    'mês': [ # Usando o nome original da coluna da imagem
        '01/01/2025',
        '01/02/2025',
        '01/03/2025',
        '01/04/2025',
        '01/10/2024',
        '01/11/2024',
        '01/12/2024',
        '01/05/2025',

    ],
    'Qtd': [ # Usando o nome original da coluna da imagem
        249,
        122,
        148,
        269,
        220,
        210,
        281,
        104
    ]
}

# Nome exato da coluna que contém as datas a serem formatadas
coluna_data_para_formatar = 'mês'

# --- Execução Principal ---
logging.info("-----------------------------------------------------")
logging.info(f"Criando DataFrame interno para gerar '{nome_arquivo_csv_destino}'.")
logging.info("Formatando datas para AAAA-MM-DD e salvando como CSV.")
logging.info("-----------------------------------------------------")

# 1. Garante que a pasta de destino do CSV exista
try:
    # Cria a pasta (e subpastas) se não existirem
    os.makedirs(caminho_pasta_destino_csv, exist_ok=True)
    logging.info(f"[OK] Pasta de destino CSV verificada/criada: '{caminho_pasta_destino_csv}'")
except Exception as e:
    logging.error(f"!!! ERRO FATAL: Falha ao verificar/criar pasta de destino: {e}")
    exit() # Interrompe se não conseguir criar a pasta

# 2. Cria o DataFrame a partir dos dados definidos
try:
    df_processed = pd.DataFrame(dados)
    logging.info(f"DataFrame criado com {len(df_processed)} linhas.")

    # 3. Formata a Coluna de Data ('mês') para AAAA-MM-DD
    if coluna_data_para_formatar in df_processed.columns:
        logging.info(f"Formatando coluna '{coluna_data_para_formatar}' para AAAA-MM-DD...")
        try:
            # Converte a coluna para datetime, especificando o formato de origem DD/MM/YYYY
            # errors='coerce' transforma erros de conversão em NaT (Not a Time)
            datas_convertidas = pd.to_datetime(df_processed[coluna_data_para_formatar], format='%d/%m/%Y', errors='coerce')

            # Formata as datas convertidas com sucesso para AAAA-MM-DD como string
            df_processed[coluna_data_para_formatar] = datas_convertidas.dt.strftime('%Y-%m-%d')

            # Verifica se houve erros na conversão original
            erros_conversao = datas_convertidas.isna().sum()
            if erros_conversao > 0:
                logging.warning(f"Coluna '{coluna_data_para_formatar}': {erros_conversao} valores não eram datas válidas no formato DD/MM/YYYY e ficarão vazios.")
                # Preenche os erros (que viraram NaN no strftime) com vazio ''
                df_processed[coluna_data_para_formatar].fillna('', inplace=True)

            logging.info(f"Coluna '{coluna_data_para_formatar}' formatada.")
        except Exception as e:
             logging.error(f"Erro ao formatar coluna de data '{coluna_data_para_formatar}': {e}")
             df_processed[coluna_data_para_formatar] = '' # Define como vazio em caso de erro geral
    else:
        logging.warning(f"Coluna de data '{coluna_data_para_formatar}' não encontrada no DataFrame criado.")

    # 4. Opcional: Garante que a coluna 'Qtd' seja numérica (se necessário)
    coluna_quantidade = 'Qtd'
    if coluna_quantidade in df_processed.columns:
        logging.info(f"Verificando/Convertendo coluna '{coluna_quantidade}' para numérico...")
        df_processed[coluna_quantidade] = pd.to_numeric(df_processed[coluna_quantidade], errors='coerce')
        # Preenche erros de conversão numérica com 0 (ou pode usar pd.NA)
        df_processed[coluna_quantidade].fillna(0, inplace=True)


    # 5. Salva como CSV
    logging.info(f"Salvando dados processados em CSV: '{caminho_completo_csv_destino}'...")
    # Garante que outros nulos (se houver) virem strings vazias
    df_processed.fillna('', inplace=True)
    df_processed.to_csv(caminho_completo_csv_destino,
                        sep=',',                 # Separador vírgula
                        encoding='utf-8-sig',    # Encoding bom para Excel ler acentos
                        index=False,             # Não salva índice do pandas
                        quotechar='"',
                        quoting=csv.QUOTE_MINIMAL) # Aspas só quando necessário
    logging.info(f"✅ Arquivo CSV '{nome_arquivo_csv_destino}' criado com sucesso em '{caminho_pasta_destino_csv}'.")


except ValueError as ve:
     logging.error(f"!!! ERRO ao criar DataFrame: {ve}.")
except PermissionError:
    logging.error(f"!!! ERRO DE PERMISSÃO ao salvar CSV em '{caminho_completo_csv_destino}'. Verifique se o arquivo está aberto ou se há permissão de escrita na pasta.")
except Exception as e:
    logging.error(f"!!! ERRO GERAL INESPERADO: {e}")
    import traceback
    traceback.print_exc()

logging.info("--- FIM DO SCRIPT ---")

2025-06-07 14:48:58,452 - INFO - -----------------------------------------------------
2025-06-07 14:48:58,453 - INFO - Criando DataFrame interno para gerar 'CorteCaptacao.csv'.
2025-06-07 14:48:58,453 - INFO - Formatando datas para AAAA-MM-DD e salvando como CSV.
2025-06-07 14:48:58,454 - INFO - -----------------------------------------------------
2025-06-07 14:48:58,456 - INFO - [OK] Pasta de destino CSV verificada/criada: 'C:\Users\Florestas\Desktop\Relatorio Oficial 2025\Excel - Pedidos\Pedidos'
2025-06-07 14:48:58,458 - INFO - DataFrame criado com 8 linhas.
2025-06-07 14:48:58,459 - INFO - Formatando coluna 'mês' para AAAA-MM-DD...
2025-06-07 14:48:58,467 - INFO - Coluna 'mês' formatada.
2025-06-07 14:48:58,468 - INFO - Verificando/Convertendo coluna 'Qtd' para numérico...
C:\Users\Florestas\AppData\Local\Temp\ipykernel_1308\2663631126.py:97: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behav

# Divergancia na Separação

In [14]:
import pandas as pd
import os
import logging
import csv
import re
import unicodedata

# --- Configurações ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Caminhos e Nomes de Arquivo ---
# Pasta onde está o Excel de origem
caminho_pasta_origem_excel = r'C:\Users\Florestas\Desktop\Relatorio Oficial 2025\Excel - Pedidos'
# !!! NOVO ARQUIVO DE ORIGEM !!!
nome_arquivo_excel_origem = 'Faturado Não Enviado (HUB).xlsx'
caminho_completo_excel_origem = os.path.join(caminho_pasta_origem_excel, nome_arquivo_excel_origem)

# !!! NOME EXATO DA ABA/PLANILHA DENTRO DO ARQUIVO EXCEL !!!
nome_da_aba_excel = 'Divergencia por Colaborador'

# ====> DEFINA O DESTINO E NOME DO ARQUIVO FINAL <====
# Onde salvar o CSV? Qual nome dar a ele?
# Substitua os valores abaixo pelos corretos para este arquivo.

# Exemplo 1: Pasta Local 'Pedidos'
caminho_pasta_destino_csv = r'C:\Users\Florestas\Desktop\Relatorio Oficial 2025\Excel - Pedidos\Pedidos'
nome_arquivo_csv_destino = 'Faturado_Nao_Enviado_Divergencia.csv' # Nome sugerido

# Exemplo 2: Pasta Google Drive 'Custos Transportadora'
# caminho_pasta_destino_csv = r'G:\Meu Drive\Relatorio Oficial 2025\Drive\Custos Transportadora'
# nome_arquivo_csv_destino = 'Faturado_Nao_Enviado_Divergencia.csv' # Nome sugerido

# Verifique se o caminho e nome abaixo estão corretos para este arquivo!
caminho_completo_csv_destino = os.path.join(caminho_pasta_destino_csv, nome_arquivo_csv_destino)
# ======================================================

# --- Funções Auxiliares ---
def limpar_nome_coluna(col_name):
    # (Mesma função para limpar nomes de colunas)
    if not isinstance(col_name, str): col_name = str(col_name)
    nfkd_form = unicodedata.normalize('NFKD', col_name)
    ascii_name = nfkd_form.encode('ASCII', 'ignore').decode('ASCII')
    lower_name = ascii_name.lower()
    cleaned_name = re.sub(r'\s+', '_', lower_name)
    cleaned_name = re.sub(r'[^\w_]+', '', cleaned_name)
    cleaned_name = re.sub(r'_+', '_', cleaned_name)
    cleaned_name = cleaned_name.strip('_')
    if cleaned_name and cleaned_name[0].isdigit(): cleaned_name = '_' + cleaned_name
    if not cleaned_name: cleaned_name = 'coluna_vazia'
    return cleaned_name

# --- Execução Principal ---
logging.info("-----------------------------------------------------")
logging.info(f"Processando ABA '{nome_da_aba_excel}' de '{nome_arquivo_excel_origem}'")
logging.info(f"Para gerar o arquivo CSV '{nome_arquivo_csv_destino}'.")
logging.info("-----------------------------------------------------")

# 1. Garante que a pasta de destino do CSV exista
try:
    os.makedirs(caminho_pasta_destino_csv, exist_ok=True)
    logging.info(f"[OK] Pasta de destino CSV verificada/criada: '{caminho_pasta_destino_csv}'")
except Exception as e:
    logging.error(f"!!! ERRO FATAL: Falha ao verificar/criar pasta de destino: {e}")
    exit()

# 2. Verifica se o arquivo Excel de origem existe
if not os.path.exists(caminho_completo_excel_origem):
    logging.error(f"!!! ERRO FATAL: Arquivo Excel de ORIGEM não encontrado: '{caminho_completo_excel_origem}'.")
    exit()

# 3. Lê a ABA ESPECÍFICA do arquivo Excel
df_processed = None
try:
    logging.info(f"Lendo ABA '{nome_da_aba_excel}' do arquivo Excel: {nome_arquivo_excel_origem}...")
    # Usa sheet_name para especificar qual aba ler
    df_processed = pd.read_excel(caminho_completo_excel_origem, sheet_name=nome_da_aba_excel, dtype=str)
    if df_processed is None or df_processed.empty: raise ValueError(f"Aba '{nome_da_aba_excel}' está vazia ou falha na leitura.")
    logging.info(f"Leitura da aba OK: {len(df_processed)} linhas.")

    # --- !!! ADICIONE AQUI OS PASSOS DE PROCESSAMENTO NECESSÁRIOS !!! ---
    #      (Limpar N/A? Formatar Datas? Remover Duplicatas? Selecionar Colunas?)

    # Exemplo: Limpeza básica de espaços (geralmente útil)
    logging.info("Limpando espaços extras...")
    for col in df_processed.columns:
        if pd.api.types.is_object_dtype(df_processed[col]):
            df_processed[col] = df_processed[col].str.strip()

    # Exemplo: Limpar Nomes das Colunas (geralmente útil)
    logging.info("Limpando nomes das colunas...")
    df_processed.columns = [limpar_nome_coluna(col) for col in df_processed.columns]
    logging.info(f"Nomes das colunas limpos (ex: {list(df_processed.columns[:5])}...).")
    # -------------------------------------------------------------------

    # 4. Salva como CSV (Substituindo o CSV anterior no destino)
    if not df_processed.empty:
        logging.info(f"Salvando dados processados em CSV: '{caminho_completo_csv_destino}'...")
        df_processed.fillna('', inplace=True) # Garante que nulos virem strings vazias
        df_processed.to_csv(caminho_completo_csv_destino, sep=',', encoding='utf-8-sig', index=False, quotechar='"', quoting=csv.QUOTE_MINIMAL)
        logging.info(f"✅ Arquivo CSV '{nome_arquivo_csv_destino}' criado/atualizado com sucesso em '{caminho_pasta_destino_csv}'.")
    else:
         logging.warning("DataFrame ficou vazio após leitura/processamento. Nenhum CSV salvo.")

except FileNotFoundError:
    logging.error(f"!!! ERRO: Arquivo Excel não encontrado em '{caminho_completo_excel_origem}'.")
except ValueError as ve:
     # Erro pode acontecer se a ABA não existir
     logging.error(f"!!! ERRO: {ve}. Verifique se a aba '{nome_da_aba_excel}' existe no arquivo Excel.")
except PermissionError:
    logging.error(f"!!! ERRO DE PERMISSÃO ao salvar CSV em '{caminho_completo_csv_destino}'.")
except Exception as e:
    logging.error(f"!!! ERRO GERAL INESPERADO: {e}")
    import traceback
    traceback.print_exc()

logging.info("--- FIM DO SCRIPT ---")

2025-05-28 13:04:44,562 - INFO - -----------------------------------------------------
2025-05-28 13:04:44,562 - INFO - Processando ABA 'Divergencia por Colaborador' de 'Faturado Não Enviado (HUB).xlsx'


2025-05-28 13:04:44,562 - INFO - Para gerar o arquivo CSV 'Faturado_Nao_Enviado_Divergencia.csv'.
2025-05-28 13:04:44,562 - INFO - -----------------------------------------------------
2025-05-28 13:04:44,570 - INFO - [OK] Pasta de destino CSV verificada/criada: 'C:\Users\Florestas\Desktop\Relatorio Oficial 2025\Excel - Pedidos\Pedidos'
2025-05-28 13:04:44,571 - INFO - Lendo ABA 'Divergencia por Colaborador' do arquivo Excel: Faturado Não Enviado (HUB).xlsx...
2025-05-28 13:04:44,872 - INFO - Leitura da aba OK: 219 linhas.
2025-05-28 13:04:44,872 - INFO - Limpando espaços extras...
2025-05-28 13:04:44,879 - INFO - Limpando nomes das colunas...
2025-05-28 13:04:44,881 - INFO - Nomes das colunas limpos (ex: ['pedido', 'loja_pedido', 'nome']...).
2025-05-28 13:04:44,881 - INFO - Salvando dados processados em CSV: 'C:\Users\Florestas\Desktop\Relatorio Oficial 2025\Excel - Pedidos\Pedidos\Faturado_Nao_Enviado_Divergencia.csv'...
2025-05-28 13:04:44,881 - INFO - ✅ Arquivo CSV 'Faturado_Nao_E

## Sac - faturado e n enviado

In [15]:
import pandas as pd
import os
import glob
import re
import unicodedata
import logging

# --- Configurações Iniciais ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Constantes e Configurações ---
PASTA_DOWNLOADS = os.path.join(os.path.expanduser('~'), 'Downloads')

# --- DESTINO CORRIGIDO PARA A PASTA 'Pedidos' ---
PASTA_DESTINO_EXCEL = r'C:\Users\Florestas\Desktop\Relatorio Oficial 2025\Excel - Pedidos\Pedidos' # << CORRIGIDO AQUI
# ------------------------------------------------

# Mantendo o nome do arquivo de destino como .xlsx
NOME_ARQUIVO_EXCEL_DESTINO = 'sac_faturado_nao_enviado_processado.xlsx'
CAMINHO_COMPLETO_EXCEL_DESTINO = os.path.join(PASTA_DESTINO_EXCEL, NOME_ARQUIVO_EXCEL_DESTINO)

# Nomes das colunas de data no Excel ORIGINAL e como queremos que fiquem no arquivo final
MAPEAMENTO_COLUNAS_DATAS = {
    'Data/Hora criação': 'data_abertura',
    'Data/Hora finalização': 'data_fechamento'
}

# Outras colunas para renomear e limpar
MAPEAMENTO_OUTRAS_COLUNAS = {
    'Situação': 'situacao', 'Cód Atendimento': 'cod_atendimento', 'Nome Pessoa': 'nome_pessoa',
    'Cód Pedido': 'cod_pedido', 'Cód Externo Pedido': 'cod_externo_pedido', 'cod_pessoa': 'cod_pessoa',
    'CPF': 'cpf', 'Sexo': 'sexo', 'Idade': 'idade', 'RuaResidencial': 'rua_residencial',
    'ComplementoResidencial': 'complemento_residencial', 'BairroResidencial': 'bairro_residencial',
    'CidadeResidencial': 'cidade_residencial', 'EstadoResidencial': 'estado_residencial',
    'CEPResidencial': 'cep_residencial', 'E-mail principal': 'email_principal',
    'Cód Atendente Criação': 'cod_atendente_criacao', 'Nome do Atendente Criação': 'nome_atendente_criacao',
    'Cód Atendente Finalização': 'cod_atendente_finalizacao', 'Nome do atendente Finalização': 'nome_atendente_finalizacao',
    'Origem': 'origem', 'Tipo': 'tipo', 'Cód Produto': 'cod_produto', 'Nome do Produto': 'nome_produto',
    'Número do lote': 'numero_lote', 'Cód Transportadora': 'cod_transportadora',
    'Nome Transportadora': 'nome_transportadora', 'Pergunta': 'pergunta', 'Resposta': 'resposta'
}

def limpar_nome_coluna(nome_coluna):
    """Limpa um nome de coluna: minúsculas, sem acentos, espaços/especiais para underscore."""
    if not isinstance(nome_coluna, str): nome_coluna = str(nome_coluna)
    nfkd_form = unicodedata.normalize('NFKD', nome_coluna)
    nome_sem_acentos = "".join([c for c in nfkd_form if not unicodedata.combining(c)])
    nome_minusculo = nome_sem_acentos.lower()
    nome_limpo = re.sub(r'[^\w_]+', '_', nome_minusculo)
    nome_limpo = re.sub(r'_+', '_', nome_limpo)
    nome_limpo = nome_limpo.strip('_')
    if nome_limpo and nome_limpo[0].isdigit(): nome_limpo = '_' + nome_limpo
    return nome_limpo if nome_limpo else 'coluna_indefinida'

def processar_arquivo_excel_para_xlsx():
    """
    Encontra o arquivo Excel (.xls) mais recente, lê, processa e salva como um novo arquivo .xlsx.
    """
    logging.info("--- INICIANDO PROCESSAMENTO PARA SALVAR COMO .XLSX ---")
    logging.info(f"Pasta de destino configurada para: {PASTA_DESTINO_EXCEL}") # Log mostrará o caminho correto

    # 1. Encontrar o arquivo Excel de origem
    logging.info(f"Procurando arquivos 'Atendimen*.xls' em '{PASTA_DOWNLOADS}'...")
    arquivos_encontrados = glob.glob(os.path.join(PASTA_DOWNLOADS, "Atendimen*.xls"))
    if not arquivos_encontrados:
        logging.error("Nenhum arquivo Excel 'Atendimen*.xls' encontrado.")
        return
    arquivo_excel_origem = max(arquivos_encontrados, key=os.path.getmtime)
    logging.info(f"Arquivo Excel de origem: '{arquivo_excel_origem}'")

    # 2. Ler o arquivo Excel de origem
    df = None
    try:
        df = pd.read_excel(arquivo_excel_origem, sheet_name=0, dtype=str, engine='xlrd')
        logging.info(f"Lidas {len(df)} linhas e {len(df.columns)} colunas do Excel com engine='xlrd'.")
    except Exception as e_xlrd:
        logging.warning(f"Falha ao ler com xlrd: {e_xlrd}")
        if "Excel xlsx file; not supported" in str(e_xlrd) or "Unsupported format, or corrupt file" in str(e_xlrd) :
            logging.info("Tentando ler como .xlsx (engine=openpyxl)...")
            try:
                df = pd.read_excel(arquivo_excel_origem, sheet_name=0, dtype=str, engine='openpyxl')
                logging.info(f"Lidas {len(df)} linhas e {len(df.columns)} colunas com engine='openpyxl'.")
            except Exception as e_openpyxl: logging.error(f"Erro ao ler como .xlsx também: {e_openpyxl}"); return
        else: logging.error(f"Erro não relacionado a formato ao ler com xlrd: {e_xlrd}"); return

    if df is None or df.empty:
        logging.warning("O DataFrame está vazio após as tentativas de leitura."); return

    # 3. Renomear e Limpar Colunas
    novos_nomes_colunas = {}
    colunas_datas_processadas = []
    for nome_col_original_excel in df.columns:
        nome_original_strip = str(nome_col_original_excel).strip()
        novo_nome_final = None
        if nome_original_strip in MAPEAMENTO_COLUNAS_DATAS:
            novo_nome_final = MAPEAMENTO_COLUNAS_DATAS[nome_original_strip]
            colunas_datas_processadas.append(novo_nome_final)
        elif nome_original_strip in MAPEAMENTO_OUTRAS_COLUNAS:
            novo_nome_final = MAPEAMENTO_OUTRAS_COLUNAS[nome_original_strip]
        else: novo_nome_final = limpar_nome_coluna(nome_original_strip)
        novos_nomes_colunas[nome_col_original_excel] = novo_nome_final
    df.rename(columns=novos_nomes_colunas, inplace=True)
    df.columns = [limpar_nome_coluna(col) for col in df.columns]
    colunas_datas_processadas = [limpar_nome_coluna(col) for col in colunas_datas_processadas]
    logging.info(f"Nomes de colunas após renomeação e limpeza final: {list(df.columns)}")

    # 4. Converter Colunas de Data
    for nome_col_data_limpo in colunas_datas_processadas:
        if nome_col_data_limpo in df.columns:
            logging.info(f"Processando coluna de data: '{nome_col_data_limpo}'")
            try:
                df[nome_col_data_limpo] = pd.to_datetime(df[nome_col_data_limpo], errors='coerce')
                if not df[nome_col_data_limpo].isna().all():
                     logging.info(f"Formatando '{nome_col_data_limpo}' para YYYY-MM-DD.")
                     df[nome_col_data_limpo] = df[nome_col_data_limpo].dt.strftime('%Y-%m-%d')
                     df[nome_col_data_limpo].replace('NaT', '', inplace=True)
                else:
                    logging.warning(f"Conversão para datetime falhou para a coluna '{nome_col_data_limpo}'. Preenchendo com vazio.")
                    df[nome_col_data_limpo] = ''
            except Exception as e:
                logging.error(f"Erro ao converter data da coluna '{nome_col_data_limpo}': {e}")
                df[nome_col_data_limpo] = ''
        else:
            logging.warning(f"Coluna de data '{nome_col_data_limpo}' não encontrada para conversão.")

    # 5. Garantir que a pasta de destino exista e Salvar como .XLSX
    try:
        os.makedirs(PASTA_DESTINO_EXCEL, exist_ok=True) # Usa o caminho corrigido
        logging.info(f"Tentando salvar arquivo Excel em: '{CAMINHO_COMPLETO_EXCEL_DESTINO}'") # Usa o caminho corrigido
        df.fillna('', inplace=True)
        df.to_excel(CAMINHO_COMPLETO_EXCEL_DESTINO, index=False, engine='openpyxl') # Usa o caminho corrigido
        logging.info(f"✅ Arquivo Excel (.xlsx) salvo com sucesso em: '{CAMINHO_COMPLETO_EXCEL_DESTINO}'") # Usa o caminho corrigido

    except PermissionError:
        logging.error(f"ERRO DE PERMISSÃO ao salvar em '{CAMINHO_COMPLETO_EXCEL_DESTINO}'.")
        logging.error("Verifique se o arquivo está aberto ou se você tem permissão para escrever nesta pasta.")
    except Exception as e:
        logging.error(f"Erro desconhecido ao criar pasta de destino ou salvar o arquivo Excel: {e}")

    logging.info("--- PROCESSAMENTO CONCLUÍDO ---")

# --- Executar o Processamento ---
if __name__ == "__main__":
    processar_arquivo_excel_para_xlsx()

2025-05-28 13:04:44,918 - INFO - --- INICIANDO PROCESSAMENTO PARA SALVAR COMO .XLSX ---
2025-05-28 13:04:44,920 - INFO - Pasta de destino configurada para: C:\Users\Florestas\Desktop\Relatorio Oficial 2025\Excel - Pedidos\Pedidos
2025-05-28 13:04:44,920 - INFO - Procurando arquivos 'Atendimen*.xls' em 'C:\Users\Florestas\Downloads'...
2025-05-28 13:04:44,930 - INFO - Arquivo Excel de origem: 'C:\Users\Florestas\Downloads\Atendimento_2d01d30b-f93e-4441-968d-2c4e8373e53c.xls'
2025-05-28 13:04:44,942 - INFO - Lidas 100 linhas e 18 colunas do Excel com engine='xlrd'.
2025-05-28 13:04:44,942 - INFO - Nomes de colunas após renomeação e limpeza final: ['cod_atendimento', 'cod_pedido', 'cod_externo_pedido', 'data_abertura', 'cod_pessoa', 'nome_pessoa', 'origem', 'tipo', 'situacao', 'regra_de_notificacao', 'nivel_atual', 'nota_fiscal', 'data_situacao', 'cod_usuario', 'nome_usuario', 'cod_usuario_criacao', 'nome_usuario_criacao', 'outras_informacoes']
2025-05-28 13:04:44,942 - INFO - Tentando sa

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
